In [1]:
from brian2 import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt, windows
from scipy.stats import linregress
%matplotlib inline
import scipy.io
from scipy.signal import resample, freqz
from scipy.interpolate import interp1d
from scipy.signal import remez, firwin, lfilter
# from factor_analyzer import FactorAnalyzer
from sklearn.decomposition import PCA
import random
import matplotlib.cm as cm
import os
import pandas as pd
from scipy.signal import csd, detrend
import pickle
from scipy.fft import fft, fftfreq
import sys
import scipy.ndimage
from brian2 import prefs
prefs.codegen.target = "numpy" # slower but I don't need to install many dependencies at least
plt.style.use('_classic_test_patch') # Plotting style

In [2]:
# Used to convert a string variable (used as input from the iteration script) to a boolean variable (when calling this script with specific inputs as strings)
def str_to_bool(s):
    if s == "True":
        return True
    elif s == "False":
        return False
    else:
        return False

In [32]:
### PARAMETERS ###
start_scope() # Re-initialize Brian

### GENERAL PARAMETERS ###########################################################
sim_name = "SIM_NAME" # str(os.getenv('simulation_name')) # < to use when iterating the simulation via another script # '___TEST_excit_distrib_small1_large1_relationship1' 
sim_method = 'euler' # 'exact' 'euler' #'euler' is less precise but faster, and seems to be good enough

# TIME PARAMETERS
fsamp = 1000  # set your fsamp # This is NOT the dt at which the simulation runs. The simulation timesteps are 0.1ms in duration by default
window_beginning_ignore = 1 # in s
window_end_ignore = 1 # in s
ISI_threshold_for_discontinuity = 0.2 # in s ; motoneurons whose max(ISI)>threshold will be removed from analysis (so only continuous MNs are kept)
# ISI_threshold_for_RT = 0.5 # in s ; the recruitment threshold of motoneurons is calculated as the force (in % MVC) at the time of the first spike whose ISI is < threshold

# VOLTAGE THRESHOLDS OF ALL NEURONS
voltage_rest = 0 * mvolt # arbitrary ; 0 at rest
voltage_thresh = 10 * mvolt # arbitrary ; 10 for generating a spike

# REVERSAL/EQUILIBRIUM POTENTIAL OF LEAK CHANNELS, EXCITATORY CHANNELS, INHIBITORY CHANNELS
E_leak = voltage_rest
# Reversal potentials (relative to resting potential) from Elias & Kohn 2013 are 70 and -16 for xcitatory and inhibitory, respectively.
# They corrspond roughly to the reversal potentials of sodium channels (for E_excit) and chloride channels (for E_inhib).
# However, to make everything easier to work with, I tried to make the inhibition and excitation have roughly equivalent net effects on firing rates by making the reversal potential symmetric relative to half the firing threshold.
E_excit = ((voltage_thresh + voltage_rest)/2) + 20 * mvolt # Reversal potential for excitatory input
E_inhib = ((voltage_thresh + voltage_rest)/2) - 20 * mvolt  # Reversal potential for inhibitory input

# NUMBER OF NEURONS SIMULATED
nb_motoneurons_full_pool = 300 # All motoneurons from the pool to be simulated
# Rough approximation of the number of motor units found in the tibialis anterior in humans (Motor Unit - Heckamn & Enoka 2012, ref 220 & 694)
# Twitch torque data also comes from the tibialis anterior, so this hopefully allows for a realistic simulation of the motor pool behavior for a given force level

# SIMULATING THE HD-EMG MU IDENTIFICATION PROCESS BY SUB-SAMPLING THE ACTIVE MUs
subsample_MUs_for_analysis = False # True
nb_of_MUs_to_subsample = 50
motor_unit_subsampling_probability_distribution = 'size' # 'size' #'uniform', 'size' # if 'uniform", every active MU will have the same probability to be selected for the analysis; if 'size", larger motor units will have a higher probability of being selected
bias_towards_larger_motor_neurons_temperature = 5.0 # only used if motor_unit_subsampling_probability_distribution=='size'.
# If infinity (inf), same as uniform distribution. If ~100, probability is approximately linearly scaled according to size. Exponential bias for values below (very strong bias for temperature = 1 for example). Error if 0


### TARGET SIMULATION
target_type = 'trapezoid' #'sinusoid' # 'plateau' #'trapezoid'
target_force_level = 30 # % of the max tetanic force of the simulated MNs
# if trapezoid:
ramp_duration = 5 # in s
plateau_duration = 60
analyzis_window = 'plateau'# 'all' 'plateau' # if 'all', the entire signal will be analyzed. If 'plateau', only the plateau section will be analyzed.
# if sinusoid:
target_force_sin_freq = 1 # used only if targt_type == 'sinusoid'

### SIMULATION DURATION
if target_type != 'trapezoid':
    true_duration = 20
else:
    true_duration = ramp_duration*2 + plateau_duration
duration_with_ignored_window = (true_duration+window_beginning_ignore+window_end_ignore)*second


### INPUT PARAMETERS #########################################################

# INHIBITORY INPUT
inhibition_weight_distribution = 'multimodal' #'multimodal' 'exponential' 'mixed_additive' 'mixed_multiplicative'
# 'multimodal' = MN will receive inhibition according the weights distributed according to a normal distribution. For example, for 50-50 distribution of either 0 or 1, use...
#   # inhibition_multimodal_number_of_modes = 2
#   # inhibition_multimodal_weights_distrib_means = [0, 1]
#   # inhibition_multimodal_weights_distrib_stds = [0, 0]
inhibition_multimodal_number_of_modes = 1 # used only if 'inhibition_weight_distribution = multimodal'
inhibition_multimodal_weights_distrib_means = [0] # os.getenv('inhibition_multimodal_weights_distrib_means') # the number of element should be = to 'inhibition_multimodal_number_of_modes' # used only if 'inhibition_weight_distribution = multimodal'
    # inhibition_multimodal_weights_distrib_means = eval(inhibition_multimodal_weights_distrib_means)
inhibition_multimodal_weights_distrib_stds = [0] # os.getenv('inhibition_multimodal_weights_distrib_stds') # the number of element should be = to 'inhibition_multimodal_number_of_modes' # used only if 'inhibition_weight_distribution = multimodal'
    # inhibition_multimodal_weights_distrib_stds = eval(inhibition_multimodal_weights_distrib_stds)
inhibition_multimodal_weights_proportions = [1] # os.getenv('inhibition_multimodal_weights_proportions')
    # inhibition_multimodal_weights_proportions = eval(inhibition_multimodal_weights_proportions)
# the number of elements should be equal to 'inhibition_multimodal_number_of_modes'. If the proportions add up to more than 1, an error will occur. If they sum to less than 1, the remaining MUs not in any group will be assigned a weight of 0. # used only if 'inhibition_weight_distribution = multimodal'
# 'exponential' = MNs receiving inhibition will receive increasing or decreasing amount of inhibition according to their sizes - Martinez Valdes J physiol 2020 model = https://physoc.onlinelibrary.wiley.com/doi/full/10.1113/JP279225 
inhibition_exponential_exponent_weights = 2 # used only if 'inhibition_weight_distribution = exponential'
inhibition_exponential_constant_weights = 1.5 # used only if 'inhibition_weight_distribution = exponential'
inhibition_exponential_offset_weights = 0 # 0 # used only if 'inhibition_weight_distribution = exponential'
#   # Distribution of inhibitory weights = constant * MN_size^(exponent) + offset

nb_inhibitory_input = 0
low_pass_filter_of_inhibitory_input = 5 #in hz
inhibitory_input_mean = 30*1e-2 # float(os.getenv('inhibitory_input_mean')) < to use when iterating the simulation via another script # in millisiemens
inhibitory_input_std = 1.5*1e-2 # float(os.getenv('inhibitory_input_std'))# in millisiemens
# Determine in which order the inhibition will happen in the script
keep_force_constant_despite_inhib = True # True # False # If True, the force will be optimized taking the inhibitory input into account, so the common excitatory input will necessarily increase
# If false, the inhibitory input will be delivered after the common excitatory input has been optimized to reach the target force. So the force level will be reduced, but the common excitatory input will remain the same as when there is no inhibition.

inhibitory_input_source = 'load_synthetic_input' # 'generate_synthetic_input' ; 'load_synthetic_input'
inhibitory_input_sourcefile = 'D:/THESE/Git_Scripts/Python_Scripts/motoneuron_simulation/Synthetic_signals.csv' # used only if inhibitory_input_source == 'load_synthetic_input'
# use the N first signals (the first N columns) as inhibitory inputs, with N = nb_inhibitory_input
# Use a .csv file. The number of samples in the csv file should be >= the number of samples in the simulation
inhibitory_input_sourcefile_fsamp = 1000 # 1000 if .csv # If not matching the simulation's fsamp, the loaded signal will be upsampled or downsampled to match the simulation's sampling rate

# EXCITATORY INPUT
excitatory_input_baseline = 150*1e-2 # float(os.getenv('excitatory_input_baseline')) < to use when iterating the simulation via another script # in millisiemens # From there (this baseline value), optimizer will try to optimize in order to reach the force target # tried to tune it manually to get it close to 30% MVC already for easier optimization 
# The baseline input has the shape of the target force, but in a non-linear way. The max value corresponds to the selected baseline.
excitatory_input_std = 1.5*1e-2 # in millisiemens # Added to the "excitatory_input_baseline" learned by optimization
low_pass_filter_of_excitatory_input = 5 # in hz
excit_input_for_MVC = 600*1e-2 # in millisiemens. High input to reach a stable force output where all MNs are recruited, and with a mean firing rate of ~ 35 pps.
# This is similar to the MVC data reported in "Oya T, Riek S, Cresswell AG. Recruitment and rate coding organisation for soleus motor units across entire range of voluntary isometric plantar flexions. J Physiol 587: 4737-4748, 2009."
# ^ as found in Enoka Heckman Motor Unit 2012 (figure 14)
# For the selected contractile and electrophysiological properties, 350 MUs, and an excitatory input of 4 milliSiemens, we get a MVC torque of ~30N/m.
# The MU data (number of MUs in the pool and twitch force) comes from human tibialis anterior, so the resulting MVC torque is quite representative => see https://bmcmusculoskeletdisord.biomedcentral.com/articles/10.1186/1471-2474-14-104/tables/1 (Moraux et al 2013)
excitation_bias = 0.5
excitation_weight_smallest_MN = 1 - excitation_bias # float(os.getenv('excitation_weight_smallest_MN'))
excitation_weight_largest_MN = 1 + excitation_bias # float(os.getenv('excitation_weight_largest_MN'))
excitation_weight_relationship_from_smallest_to_largest = 1 # float(os.getenv('excitation_weight_relationship_from_smallest_to_largest')) # 2 # 1 for linear, < 1 for convex curve, > 1 for concave curve (or opposite if excitation_weight_smallest_MN > excitation_weight_largest_MN) # needs to be > 0
# https://www.desmos.com/calculator/pgb3pkffsf = check curve of distribution
# weight of 0.7 for smallest MN and weight of 1.3 for largest MN = ratio of excitatory input necessary for RT (max/min) of 8.4
# weight of 1 for smallest MN and weight of 1 for largest MN = ratio of excitatory input necessary for RT (max/min) of 13.6
# in the litterature around a 10-fold range is rapported for a given injected input (so no different weights across MNs) = Heckman & Enoka 2012 motor unit comprehensive physiology
# ratio of 0.5 for smallest and 1.5 for largest = ratio of excitatory input necessary for RT (max/min) slightly > 5

excitatory_input_source = 'load_synthetic_input' # 'generate_synthetic_input' ; 'load_synthetic_input'
excitatory_input_sourcefile = 'D:/THESE/Git_Scripts/Python_Scripts/motoneuron_simulation/Synthetic_signals.csv' # used only if excitatory_input_source == 'load_synthetic_input' or 'load_experimental_data'
# Use the last signal (last column) as common noise
# If 'load_gaussian_noise', use a .csv file. The number of samples in the csv file should be >= the number of samples in the simulation
# If 'load_experimental_data', use one .mat file or several .mat files. If several .mat files are selected, they will be concatenated. The number of samples in the (concatenated) file(s) should be >= the number of samples in the simulation
excitatory_input_sourcefile_fsamp = 1000 # 1000 if .csv ; 2048 if .mat # If not matching the simulation's fsamp, the loaded signal will be upsampled or downsampled to match the simulation's sampling rate

# INDEPENDENT NOISE
low_pass_filter_of_independent_noise = 50 # in hz
independent_noise_ratio_std = 2 # the value of noise in std (mean of 0)
# => if 2, independent input std corresponds to 2x inhibitory and excitatory input std, to get a ratio of common input = 1/3 of independnt input (Farina & Negro 2015)



### MOTONEURON PROPERTIES ##########################

# DISTRIBUTION OF MOTONEURON SIZES
min_soma_diameter = 50 # in micrometers, for smallest MN
max_soma_diameter = 100 # in micrometers, for largest MN
# Assuming that soma diameter from human motoneurons vary between 50 and 100 micrometers, loosely based on https://journals.physiology.org/doi/full/10.1152/physiol.00021.2018 (mean diameter of humans MN estimated to be ~60 micrometers)
    # ^ "Scaling of motoneurons, From Mouse to Human" Manuel et al. Physiology (2018)
# Parameter to create an exponetially decreasing dsitribution curve, with larger motoneurons being less numerous than smaller motoneurons
# Somewhat fitting the curve in Principles of Neural Science 2021 edition, Enoka chapter on motor units, fig 31-3.A
size_distribution_exponent = 2
# between 0-1 => more large MN than small MN; 1 => uniform distribution (linear relationship between MN index and soma diameter); >1 => more small motoneurons than laarge MNs
# VIsualize distribution for different min and max soma diameters, and different exponents = https://www.desmos.com/calculator/zy3ywcz4tn 

#### ELECTROPHYSIOLOGICAL MN PROPERTIES #####
# Electrophysiological properties calculated from Caillet et al 2022 https://elifesciences.org/articles/76489
# RESISTANCE - OHMS
# The resistance decreases the leak conductance and increases the weight of the excitatory and inhibitory input received by the motor neuron (=> higher resistance means higher sensitivity to input)
resistance_constant = 9.6*(10**5) # Caillet et al 2022 # in Ohms
resistance_exponent = 2.4*(-1) # Caillet et al 2022 # in Ohms
# https://www.desmos.com/calculator/pbs97zynff = visualize the curve for resistance (ohms) and input weights (between 0 and 1)
# Min resistance for smallest MN (50 micrometers) = ~80 ohms
# Max resistance for biggest MN (100 micrometers) = ~15 ohms
#### Input weight = normalized resistance, so that the input to the smallest MN is scaled by a factor of 1 #####
# Min input weight for smallest MN (50 micrometers) = 1
# Max input weight for biggest MN (100 micrometers) = ~0.19
# CONDUCTANCE - SIEMENS
membrane_conductance_scaling = 1 # membrane conductance is 1/resistance, multiplied by a scalar value (tuned by hand) to get realistic behavior of MN pool
# RHEOBASE - AMPERES
# The rheobase is modeled as an offset to the change in excitatory conductance caused by the exitatory input (clamped to 0 to avoid the excitatory input to have an hyperpolarization effect)
rheobase_constant = 9.0*(10**-4) # Caillet et al 2022 # in nanoAmps
rheobase_exponent = 2.5 # Caillet et al 2022 # in nanoAmps
rheobase_scaling = 100 # float(os.getenv('rheobase_scaling')) # < to use when iterating the simulation via another script # 100 # scalar value to multiply the rheobase by, tuned to get realistic behavior according to the arbitrary values used (such as voltage threshold and reversal potentials)
# CAPACITANCE - FARADS
capacitance_constant = 1.2 # Caillet et al 2022
capacitance_exponent = 1 # Caillet et al 2022
# AFTERHYPERPOLARIZATION DURATION & REFRACTORY PERIOD - SECONDS
# Refractory period (Caillet's paper gives equations for AHP duration but not for refractory period duration) #
# Since we are using a simplified model, we approximate the effect of the AHP by implementing an absolute refactory period that is a fraction of the true AHP duration.
AHP_duration_constant = 2.5 * (10**4) # Caillet et al 2022
AHP_duration_exponent = 1.5 * (-1) # Caillet et al 2022
refractory_period_as_AHP_fraction = 0.2 # float(os.getenv('refractory_period_as_AHP_fraction')) # Manually tuned
    # Manuel et al. 2019 "Scaling of motor output, from Mouse to Humans"
        # "Statistical methods employed at low firing rates indicate the AHP durations of low-threshold human motoneurons, presumably type S and perhaps some type FR, are ~125–140 ms."
    # Herbert & Gandevia 1999 assume a 5ms (absolute?) refractory period
    # Lateva et at 2001 = Absolute refractory period of 3ms in muscle fibers, and relative refractory period of 10ms
    # University of Washington textbook of physiology = in a typical neuron, the absolute refractory period lasts a few ms and the relative period tens of ms

### CONTRACTILE PROPERTIES (TWITCH FORCE CAUSED BY MN SPIKES)
    # Every firing of motor units will be convolved with a kernel
    #  => The kernel is a hanning window of duration which is 2x the time to peak force, and then the duration of the "down" portion of the twitch (when the force returns to baseline) is extended by 'multiplication_of_twitch_force_down_time'
    #   # Finally, the conduction delay is added before the start of the kernel
# TWITCH TORQUE - NEWTON/METERS
    # Linear interpolation to get force produced (this is a gross simplification, as the distribution of MU properties is not linearly distributed at all)
    # Data from Principles of Neural Science 2021 edition Motor Unit chapter Enoka # Figure 31-3, values for human tibialis anterior motor units => between 0-10 mN/m for smallest MUs, ~140mN/m for largest MUs
    # Also found in Motor Unit Enoka Heckman 2012, from Van Cutsem M, Feiereisen P, Duchateau J, Hainaut K. Mechanical properties and behaviour of motor units in the tibialis anterior during voluntary contractions. Can J Appl Physiol 22: 585-597, 1997
twitch_force_range_small_MU = 5 # in milliNewton/meter
twitch_force_range_large_MU = 140 # in milliNewton/meter
# TIME TO PEAK FORCE - SECONDS
# time to peak force => linear relationship (this is a gross simplification, as the distribution of MU properties is not linearly distributed at all)
time_to_peak_twitch_force_range_small_MU = 0.08 *2 # in s # smallest MU
time_to_peak_twitch_force_range_large_MU = 0.02 *2 # in s # biggest MU
# *2 because later in the script the kernel is created with twice the tie to peak force
# Time to peak torque ranges from 20ms (0.02s) to 80-100ms (0.08-0.1s) for TA in the figure next to twitch force- Principles of Neural Science 2021 edition Motor Unit chapter Enoka # Figure 31-3, values for human tibialis anterior motor units
# Also found in Motor Unit Enoka Heckman 2012, from Van Cutsem M, Feiereisen P, Duchateau J, Hainaut K. Mechanical properties and behaviour of motor units in the tibialis anterior during voluntary contractions. Can J Appl Physiol 22: 585-597, 1997
    # Doubling the value because this is time to peak force, and kernel duration is twice that
    #    # Also some data from Shoepe et al 2003 MSSE, shortening velocity of type I fiber (time to peak force small MU = 60ms) VS type IIa fiber (time to peak force fast MU = 25ms)
    #    # https://paulogentil.com/pdf/Functional%20adaptability%20of%20muscle%20fibers%20to%20long-term%20resistance%20exercise.pdf
multiplication_of_twitch_force_down_time = 4 # This is a gross simplificaion of how the twitch force return to baseline, but overall it seems to fit the behavior of motor units
    # B. R. Botterman, G. A. Iwamoto, and W. J. Gonyea (1986) = force trace of single twitches from motor units
    # Rositsa Raikova, Piotr Krutki, Jan Celichowski (2023) => Detailed model of motor units twitch force
# MOTOR UNIT TETANIC FORCE (max force which can be produced)
# Increase in force (with each additional twtich) towards max tetanic force follows a sigmoid curve, like in Fuglevand 1993 model (see section "Force nonlinearity")
twitch_tetanus_ratio_smallest_MN = 0.2 # 0.1 # smallest MU's twitch force has its peak at 20% of the MU's max tetanic force
twitch_tetanus_ratio_largest_MN = 0.3 # largest MU's twitch force has its peak at 30% of the MU's max tetanic force
    # ^ data from the model of Nagamori et al. (2021): 'Force variability is mostly not motor noise: Theoretical implications for motor control', PLOS computational Biology (see 'twitch-tetanus ratio' section): https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1008707
    # In the paper, the range spans from 0.07 to 0.53. Here, we manually tuned these values so that the minimum and maximum are 0.2 and 0.3
    # 0.2 allows the smallest MU to reach 75% of its tetanus force around 15pps, 0.3 allow the largest MU to reach 75% of its tetanus force around 35pps
    # With 0.2 and 0.3, we get around the same mean as reported in the paper (0.23)
    # 0.23 correspond to the values reported in the cat by Brown & Loeb (https://link.springer.com/article/10.1023/A:1005687416896)
    # 0.11 for all motor units in Fuglevand 1993 model
steepness_of_twitch_to_tetanus_sigmoid = 1 # for 2.8, when twitch_sum=1, actual_force~=100% of tetanus force. However, this means that the twitch force for a single twitch is 2.8 greater than the twitch force in the parameters
# ELECTROMECHANICAL DELAY - SECONDS ; AXONAL CONDUCTION VELOCITY - METERS/SECOND
# Electromechanical delay => inverse of actional conduction velocity
# Calculated from the axonal conduction velocity relationship reported in Caillet et al 2022
# ^ multiplying by two (assuming a 0.5m axon length => so correspond to the conduction speed from MN to muscle fiber)
axonal_conduction_velocity_constant = 4.0*2 # Caillet et al 2022
axonal_conduction_velocity_exponent = 0.7 # Caillet et al 2022
low_pass_filter_force = True # Can be turned to true if fast oscillation (high frequency components) in force output. This simulates the "dampening" effect of the musculoskeletal system (tendon for example)
low_pass_filter_of_force_cutoff = 10 # in hz

### PARAMETERS FOR TESTING - reduce numbers for faster simulations
COH_calc_max_iteration_nb_per_group_size = 1000 # 10 when just testing out simulations
# More iteration for smaller group sizes, because the value obtained is very dependent upon the exact neurons selected, especially when only a few MNs are used to create the CST
# ^ the number of iteration will be "COH_calc_max_iteration_nb_per_group_size / nb_of_MUs_in_CST"
max_num_optimization_iterations = 10 # 5 for faster simulation, it seems good enough in most cases (but better results with higher numbers) # 1 for testing
stop_optimizing_if_mean_error_is_below = 0.1 # in % of MVC # Ths allows to speed up the process by stopping the iterations when the learning process is basically over
consider_only_plateau_for_cost_optimization = True # this variable will be used only if target_type = 'trapezoid'. If True, the stopping of the optimization process will happen considering only the cost on the plateau and not the ramps
adam_learning_rate = 0.025

### EQUATIONS BEING RUN BY BRIAN2
LIF_equations = Equations('''
      dv/dt = (-I_leak - I_excit - I_inhib) / C_m : volt (unless refractory)
      I_leak = g_leak*(v - E_leak) : amp
      I_excit = clip(synaptic_input_excit + I_th, -inf*nA, 0*nA) : amp
      synaptic_input_excit = (ge*(v - E_excit)) : amp
      I_inhib = gi*(v - E_inhib) : amp
      ge = input_weight * input_excit(t,i) : siemens
      gi = input_weight * input_inhib(t,i) : siemens
      g_leak : siemens
      C_m : farad
      refractory_period : second
      input_weight : 1
      I_th : amp
      ''')


In [4]:
### CREATE NEW FOLDER
new_directory = sim_name
new_filename = 'parameters.txt'

# Create the directory if it doesn't exist
if not os.path.exists(new_directory):
    os.makedirs(new_directory)
else:
    directory_n = 0
    while os.path.exists(new_directory):
        directory_n = directory_n+1
        new_directory = str(sim_name + "_iter_" + str(directory_n))
        if not os.path.exists(new_directory):
            os.makedirs(new_directory)
            break
        if directory_n > 99: # prevent infinite loop
            break
save_file_path = os.path.join(new_directory, new_filename)


In [5]:

### SAVE PARAMETERS

# Write the variables to the file
with open(save_file_path, 'w') as file:
    file.write(f"General parameters -----\n")
    file.write(f"   duration_with_ignored_window: {duration_with_ignored_window}\n")
    file.write(f"   nb_motoneurons_full_pool_per_pool: {nb_motoneurons_full_pool}\n")
    file.write(f"   ISI_threshold_for_discontinuity: {ISI_threshold_for_discontinuity}\n")
    # file.write(f"   ISI_threshold_for_RT: {ISI_threshold_for_RT}\n")
    file.write(f"Sub-sampling of motor units (simulating the hdEMG motor unit identification process) -----\n")
    file.write(f"   subsample_MUs_for_analysis: {subsample_MUs_for_analysis}\n")
    file.write(f"   nb_of_MUs_to_subsample: {nb_of_MUs_to_subsample}\n")
    file.write(f"   motor_unit_subsampling_probability_distribution: {motor_unit_subsampling_probability_distribution}\n")
    file.write(f"   bias_towards_larger_motor_neurons_temperature: {bias_towards_larger_motor_neurons_temperature}\n")
    file.write(f"Rest and threshold voltage + voltage equilibrium -----\n")
    file.write(f"   voltage_rest: {voltage_rest}\n")
    file.write(f"   voltage_thresh: {voltage_thresh}\n")
    file.write(f"   E_leak: {E_leak}\n")
    file.write(f"   E_excit: {E_excit}\n")
    file.write(f"   E_inhib: {E_inhib}\n")

    file.write(f"\n")
    file.write(f"Input parameters -----\n")
    file.write(f"   Common exitatory input -----\n")
    file.write(f"       excitatory_input_baseline (before optimized input learning): {excitatory_input_baseline}\n")
    file.write(f"       excitatory input for MVC: {excit_input_for_MVC}\n")
    file.write(f"       excitatory_input_std: {excitatory_input_std }\n")
    file.write(f"       excitatory_input_source: {excitatory_input_source }\n")
    file.write(f"       excitation_weight_smallest_MN: {excitation_weight_smallest_MN }\n")
    file.write(f"       excitation_weight_largest_MN: {excitation_weight_largest_MN }\n")
    file.write(f"       excitation_weight_relationship_from_smallest_to_largest: {excitation_weight_relationship_from_smallest_to_largest }\n")
    file.write(f"       excitatory_input_sourcefile: {excitatory_input_sourcefile }\n")
    file.write(f"       excitatory_input_sourcefile_fsamp: {excitatory_input_sourcefile_fsamp }\n")
    file.write(f"   Independent input (noise) -----\n")
    file.write(f"       low_pass_filter_of_independent_noise: {low_pass_filter_of_independent_noise}\n")
    file.write(f"       independent_noise_ratio_std: {independent_noise_ratio_std}\n")
    file.write(f"   Inhibition weights distribtuon -----\n")
    file.write(f"       inhibition_weight_distribution: {inhibition_weight_distribution}\n")
    file.write(f"        - If multimodal distribution of inhibitory input:\n")
    file.write(f"           inhibition_multimodal_number_of_modes: {inhibition_multimodal_number_of_modes}\n")
    file.write(f"           inhibition_multimodal_weights_distrib_means: {inhibition_multimodal_weights_distrib_means}\n")
    file.write(f"           inhibition_multimodal_weights_distrib_stds: {inhibition_multimodal_weights_distrib_stds}\n")
    file.write(f"           inhibition_multimodal_weights_proportions: {inhibition_multimodal_weights_proportions}\n")
    file.write(f"        - If exponential distribution of inhibitory input:\n")
    file.write(f"           inhibition_exponential_exponent_weights: {inhibition_exponential_exponent_weights}\n")
    file.write(f"           inhibition_exponential_constant_weights: {inhibition_exponential_constant_weights}\n")
    file.write(f"           inhibition_exponential_offset_weights: {inhibition_exponential_offset_weights}\n")
    file.write(f"       nb_inhibitory_input: {nb_inhibitory_input}\n")
    file.write(f"       low_pass_filter_of_inhibitory_input: {low_pass_filter_of_inhibitory_input}\n")
    file.write(f"       inhibitory_input_mean: {inhibitory_input_mean}\n")
    file.write(f"       inhibitory_input_std: {inhibitory_input_std}\n")
    file.write(f"       inhibitory_input_source: {inhibitory_input_source}\n")
    file.write(f"       inhibitory_input_sourcefile: {inhibitory_input_sourcefile}\n")
    file.write(f"       inhibitory_input_sourcefile_fsamp: {inhibitory_input_sourcefile_fsamp}\n")

    file.write(f"\n")
    file.write(f"Force target parameters -----\n")
    file.write(f"   target_type: {target_type}\n")
    file.write(f"   target_force_level: {target_force_level}% MVC\n")
    file.write(f"   If trapezoid:\n")
    file.write(f"       ramp_duration: {ramp_duration}\n")
    file.write(f"       plateau_duration: {plateau_duration}\n")
    file.write(f"   If NOT trapezoid:\n")
    file.write(f"       true_duration: {true_duration}\n")
    file.write(f"   If sisnusoid:\n")
    file.write(f"       target_force_sin_freq (used only if targt_type == 'sinusoid'): {target_force_sin_freq}\n")
    file.write(f"   low_pass_filter_force: {low_pass_filter_force}\n")
    file.write(f"   low_pass_filter_of_force_cutoff: {low_pass_filter_of_force_cutoff}\n")
    file.write(f"   keep_force_constant_despite_inhib: {keep_force_constant_despite_inhib}\n")

    file.write(f"Motor neurons size -----\n")
    file.write(f"   min_soma_diameter: {min_soma_diameter}\n")
    file.write(f"   max_soma_diameter: {max_soma_diameter}\n")
    file.write(f"   size_distribution_exponent : {size_distribution_exponent}\n")

    file.write(f"\n")
    file.write(f"Twitch force properties - parameters -----\n")
    file.write(f"   twitch_force_range_small_MU: {twitch_force_range_small_MU}\n")
    file.write(f"   twitch_force_range_large_MU: {twitch_force_range_large_MU}\n")
    file.write(f"   twitch_tetanus_ratio_smallest_MN: {twitch_tetanus_ratio_smallest_MN}\n")
    file.write(f"   twitch_tetanus_ratio_smallest_MN: {twitch_tetanus_ratio_smallest_MN}\n")
    file.write(f"   steepness_of_twitch_to_tetanus_sigmoid: {steepness_of_twitch_to_tetanus_sigmoid}\n")
    file.write(f"   time_to_peak_twitch_force_range_small_MU: {time_to_peak_twitch_force_range_small_MU/2}\n") #/2 because actual time to peak torque is half the value
    file.write(f"   time_to_peak_twitch_force_range_large_MU: {time_to_peak_twitch_force_range_large_MU/2}\n") #/2 because actual time to peak torque is half the value
    file.write(f"   multiplication_of_twitch_force_down_time: {multiplication_of_twitch_force_down_time}\n")
    file.write(f"   axonal_conduction_velocity_constant: {axonal_conduction_velocity_constant}\n")
    file.write(f"   axonal_conduction_velocity_exponent: {axonal_conduction_velocity_exponent}\n")

    file.write(f"\n")
    file.write(f"Electrophysiological properties - parameters -----\n")
    file.write(f"   resistance_constant: {resistance_constant}\n")
    file.write(f"   resistance_exponent: {resistance_exponent}\n")
    file.write(f"   membrane_conductance_scaling: {membrane_conductance_scaling}\n")
    file.write(f"   rheobase_constant: {rheobase_constant}\n")
    file.write(f"   rheobase_exponent: {rheobase_exponent}\n")
    file.write(f"   rheobase_scaling: {rheobase_scaling}\n")
    file.write(f"   capacitance_constant: {capacitance_constant}\n")
    file.write(f"   capacitance_exponent: {capacitance_exponent}\n")
    file.write(f"   AHP_duration_constant: {AHP_duration_constant}\n")
    file.write(f"   AHP_duration_exponent: {AHP_duration_exponent}\n")
    file.write(f"   refractory_period_as_AHP_fraction: {refractory_period_as_AHP_fraction}\n")



In [ ]:
# Define lerp (linear interpolation) function:
def lerp(a, b, t):
    return a + t * (b - a)

####### Generate motoneurons
motoneuron_soma_diameters = np.zeros(nb_motoneurons_full_pool)
motoneuron_normalized_soma_diameters = np.zeros(nb_motoneurons_full_pool)
for mni in range(nb_motoneurons_full_pool):
    motoneuron_soma_diameters[mni] = lerp(min_soma_diameter,max_soma_diameter, (mni/(nb_motoneurons_full_pool-1))**size_distribution_exponent )
    motoneuron_normalized_soma_diameters[mni] =lerp(0, 1, (mni/(nb_motoneurons_full_pool-1))**size_distribution_exponent )

# Plot histogram of motoneuron sizes
plt.figure()
# Create the histogram
counts, bins, patches = plt.hist(motoneuron_soma_diameters, density=True)
# Multiply the counts by 100 to convert to percentage
counts_percentage = counts * 100
# Plot the histogram again with the adjusted counts
plt.clf()  # Clear the current plot
plt.hist(motoneuron_soma_diameters, density=False, weights=np.ones_like(motoneuron_soma_diameters) * (100 / len(motoneuron_soma_diameters)),
         edgecolor='white', color='gray', alpha=1)
plt.vlines(min_soma_diameter,plt.ylim()[0],plt.ylim()[1],color='C1', label='Min soma diameter', linewidth=2)
plt.vlines(max_soma_diameter,plt.ylim()[0],plt.ylim()[1],color='C3', label='Max soma diameter', linewidth=2)
plt.legend()
plt.xlabel("Motoneuron size (soma diameter in micrometer)")
plt.ylabel("Proportion (% of total nb of motoneurons)")
plt.title("Distribution of motor neuron sizes")
new_filename = f'MN_sizes_distribution.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


plt.figure()
# Create the histogram
counts, bins, patches = plt.hist(motoneuron_normalized_soma_diameters, density=True)
# Multiply the counts by 100 to convert to percentage
counts_percentage = counts * 100
# Plot the histogram again with the adjusted counts
plt.clf()  # Clear the current plot
plt.hist(motoneuron_normalized_soma_diameters, density=False, weights=np.ones_like(motoneuron_normalized_soma_diameters) * (100 / len(motoneuron_normalized_soma_diameters)),
         edgecolor='white', color='gray', alpha=0.5)
plt.vlines(0,plt.ylim()[0],plt.ylim()[1],color='C1', label='Min soma diameter', linewidth=2)
plt.vlines(1,plt.ylim()[0],plt.ylim()[1],color='C3', label='Max soma diameter', linewidth=2)
plt.legend()
plt.xlabel("Normalized motoneuron size")
plt.ylabel("Proportion (% of total nb of motoneurons)")

plt.figure()
plt.plot(motoneuron_soma_diameters, color='gray')
plt.xlabel("Motoneuron idx")
plt.ylabel("MN size (soma diameter in micrometers)")
plt.title(f"Size of MNs according to index \n Mean soma diameter =  micrometers")
new_filename = f'MN_sizes_according_to_idx.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [7]:
# CONVOLVE TO GET FORCE
import math

def Twitch_summation_towards_tetanus(twitch_sum, tetanus_force):
    actual_force = np.zeros(len(twitch_sum))
    for sampli in range(len(twitch_sum)):
        normalized_force = 1 - (2 / (1 + math.exp(2 * twitch_sum[sampli]))) # Sigmoid function
        actual_force[sampli] = normalized_force * tetanus_force
    return actual_force

def Convolve_to_get_force(binary_spike_trains, corresponding_MN_idx, absolute_or_normalized):

    force_per_MU = []
    force_total = np.zeros(binary_spike_trains.shape[1])

    if ndim(binary_spike_trains) > 1:
        if shape(binary_spike_trains)[0] != len(corresponding_MN_idx):
            print("Error = the number of MU indices provided do not match with the number of rows in the binary matrix")
        for mni in range(shape(binary_spike_trains)[0]):
            # 'same' mode means the output length will be the same as the input length
            temp_force = np.convolve(binary_spike_trains[mni,:], twitch_convolution_window[corresponding_MN_idx[mni]]*motoneurons_twitch_to_tetanus_ratios[corresponding_MN_idx[mni]], mode='same')
            temp_force = Twitch_summation_towards_tetanus(temp_force, motoneurons_tetanus_forces[corresponding_MN_idx[mni]])
            if absolute_or_normalized == 'normalized':
                temp_force = (temp_force / max_MVC_force_absolute) * 100 # * 100 to get a percentage
            force_per_MU.append(temp_force)
            # ax.plot(force_total, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5)
            force_total = force_total + temp_force
    else:
        # if len(corresponding_MN_idx) != 1:        # len() doesn't work on a unique element
        #     print("Error = the number of MU indices provided do not match with the number of rows in the binary matrix")
        temp_force = np.convolve(binary_spike_trains, twitch_convolution_window[corresponding_MN_idx[mni]]*motoneurons_twitch_to_tetanus_ratios[corresponding_MN_idx[mni]], mode='same')
        temp_force = Twitch_summation_towards_tetanus(temp_force, motoneurons_tetanus_forces[corresponding_MN_idx[mni]])
        if absolute_or_normalized == 'normalized':
            temp_force = (temp_force / max_MVC_force_absolute) * 100 # * 100 to get a percentage
        force_per_MU.append(temp_force)
        force_total = force_total + temp_force
    return force_total

In [ ]:
motoneuron_resistances = np.zeros(nb_motoneurons_full_pool)
motoneuron_input_weights = np.zeros(nb_motoneurons_full_pool)
motoneuron_capacitances = np.zeros(nb_motoneurons_full_pool)
motoneurons_membrane_conductance = np.zeros(nb_motoneurons_full_pool)
motoneurons_AHP_durations = np.zeros(nb_motoneurons_full_pool)
motoneurons_refractory_periods = np.zeros(nb_motoneurons_full_pool)
motoneurons_rheobases = np.zeros(nb_motoneurons_full_pool)
for mni in range(nb_motoneurons_full_pool):
    motoneuron_resistances[mni] = resistance_constant*(motoneuron_soma_diameters[mni]**resistance_exponent)
    motoneuron_input_weights[mni] = motoneuron_resistances[mni] / motoneuron_resistances[0] # normalized value so that smallest MN has weight of 1
    motoneuron_capacitances[mni] = capacitance_constant*(motoneuron_soma_diameters[mni]**capacitance_exponent)
    motoneurons_membrane_conductance[mni] = 1/motoneuron_resistances[mni] * membrane_conductance_scaling
    motoneurons_AHP_durations[mni] = AHP_duration_constant*(motoneuron_soma_diameters[mni]**AHP_duration_exponent)
    motoneurons_refractory_periods[mni] = motoneurons_AHP_durations[mni] * refractory_period_as_AHP_fraction
    motoneurons_rheobases[mni] = rheobase_constant*(motoneuron_soma_diameters[mni]**rheobase_exponent)
plt.figure(figsize=(7,5))
fig, ax1 = plt.subplots()
curve1, = ax1.plot(motoneuron_resistances, label = "Resistance (ohms)", color = 'C1', linewidth = 3)
ax2 = ax1.twinx()
curve2, = ax2.plot(motoneuron_input_weights, label = "Input weight (normalized input resistance)", color = 'C5', linewidth = 2, linestyle=":")
curves = [curve1, curve2]
labels = [curve.get_label() for curve in curves]
ax1.legend(curves, labels, loc='best')
ax1.tick_params(axis='y', labelcolor='C1')
ax1.set_ylabel("Resistance (ohms)", color ='C1')
ax2.tick_params(axis='y', labelcolor='C5')
ax2.set_ylabel("Normalized input resistance (weight between 0 and 1)", color ='C5')
plt.figure(figsize=(7,5))
plt.plot(motoneuron_capacitances, label = "Capacitance (microFarads)", color = 'C2')
plt.legend()
plt.xlabel("MN index")
plt.ylabel("Capacitance (Farads)")
plt.figure(figsize=(7,5))
plt.plot(motoneurons_membrane_conductance, label = "Membrane conductance (milliSiemens)", color = 'C3')
plt.legend()
plt.xlabel("MN index")
plt.ylabel("Membrane conductance (milliSiemens)")
plt.figure(figsize=(7,5))
plt.plot(motoneurons_refractory_periods, label = "Scaled refractory period (ms) as surrogate for the AHP duration", color = 'C6')
plt.legend()
plt.xlabel("MN index")
plt.ylabel("Refractory period (ms)")
plt.figure(figsize=(7,5))
plt.plot(motoneurons_rheobases, label = "Rheobase (nanoAmperes)", color = 'C7')
plt.legend()
plt.xlabel("MN index")
plt.ylabel("Rheobase (nanoAmperes)")

# Twitch force and duration and electromechanical delay (implemented as zeros before the kernel)
twitch_force_motoneurons = np.zeros(nb_motoneurons_full_pool)
motoneurons_twitch_to_tetanus_ratios = np.zeros(nb_motoneurons_full_pool)
motoneurons_tetanus_forces = np.zeros(nb_motoneurons_full_pool)
electromechanical_delay_motoneurons = np.zeros(nb_motoneurons_full_pool)
twitch_duration_motoneurons = np.zeros(nb_motoneurons_full_pool)
twitch_convolution_window = [None] * nb_motoneurons_full_pool
for mni in range(nb_motoneurons_full_pool):
    twitch_force_motoneurons[mni] = lerp(twitch_force_range_small_MU,twitch_force_range_large_MU,motoneuron_normalized_soma_diameters[mni]) * steepness_of_twitch_to_tetanus_sigmoid
    motoneurons_twitch_to_tetanus_ratios[mni] = lerp(twitch_tetanus_ratio_smallest_MN,
                                                     twitch_tetanus_ratio_largest_MN,
                                                     motoneuron_normalized_soma_diameters[mni])
    motoneurons_tetanus_forces[mni] = twitch_force_motoneurons[mni] * (1/motoneurons_twitch_to_tetanus_ratios[mni])
    electromechanical_delay_motoneurons[mni] = 1/(axonal_conduction_velocity_constant*(motoneuron_soma_diameters[mni]**(axonal_conduction_velocity_exponent)))
    twitch_duration_motoneurons[mni] = lerp(time_to_peak_twitch_force_range_small_MU,time_to_peak_twitch_force_range_large_MU,motoneuron_normalized_soma_diameters[mni])
    # twitch_convolution_window[mni] = ((fsamp * twitch_duration_motoneurons[mni] * (1/2))**-1) * windows.hann(round(fsamp * twitch_duration_motoneurons[mni])) *  twitch_force_motoneurons[mni]
    twitch_convolution_window[mni] = windows.hann(round(fsamp * twitch_duration_motoneurons[mni]))
    # Extend the ramp down phase of the twitch so that it is five times the size of the ramp up phase (crude approximation based on Figure 1 of Raikova 2023. Full model explained in the paper https://www.sciencedirect.com/science/article/pii/S1050641123000330?via%3Dihub)
    twitch_force_down_temp = twitch_convolution_window[mni][int(np.round(len(twitch_convolution_window[mni])/2)):len(twitch_convolution_window[mni])]
    twitch_convolution_window[mni] = twitch_convolution_window[mni][:int(np.round(len(twitch_convolution_window[mni])/2))] # remove the down part (to be added in a few lines later)
    original_indices = np.linspace(0, len(twitch_force_down_temp) - 1, num=len(twitch_force_down_temp)) # Create the indices of the original and new vectors
    new_indices = np.linspace(0, len(twitch_force_down_temp) - 1, num = multiplication_of_twitch_force_down_time * len(twitch_force_down_temp) ) # Desired length of the new vector
    twitch_force_down_stretched = np.interp(new_indices, original_indices, twitch_force_down_temp) # Perform linear interpolation
    twitch_convolution_window[mni] = np.append(twitch_convolution_window[mni],twitch_force_down_stretched)
    # insert zeros corresponding to electromechanical delay
    delay_insamples = int(np.round(electromechanical_delay_motoneurons[mni]*fsamp))
    twitch_convolution_window[mni] = np.append(np.zeros(delay_insamples), twitch_convolution_window[mni])
    # Double the length of the vector with only zeros at the beginning, so that the convolution causes the force twitch to happen after each spike
    twitch_convolution_window[mni] = np.append(np.zeros(len(twitch_convolution_window[mni])),twitch_convolution_window[mni])
plt.figure(figsize=(10,5))
plt.plot(twitch_force_motoneurons, color='C1', label = 'Twitch force')
plt.plot(motoneurons_tetanus_forces, color='red', label = 'Tetanus force')
plt.plot(motoneurons_tetanus_forces*0.75, color='red', label = '75% of tetanus force', linestyle=':')
plt.ylabel("Torque (milliNewton/meter)")
plt.xlabel("Motoneuron index (smallest MN is 0 ; largest MN is "+str(nb_motoneurons_full_pool-1)+")")
plt.legend()
plt.title("MUs contractile force properties")
new_filename = f'Contractile_properties_force.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

plt.figure(figsize=(10,5))
plt.plot(twitch_duration_motoneurons*1e3*(1/2), color='C2', label = 'Time to peak force') # *(1/2) because time is expressed in full kernel duration (without extension of the "down" part yet)
plt.plot(electromechanical_delay_motoneurons*1e3, color='C4', label = 'Electromechanical delay')
plt.ylabel("Time (ms)")
plt.legend()
plt.xlabel("Motoneuron index (smallest MN is 0 ; largest MN is "+str(nb_motoneurons_full_pool-1)+")")
plt.title("MUs contractile velocty properties")
new_filename = f'Contractile_properties_velocity.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Getting a smooth color blend from a given colormap
colormap_temp = cm.get_cmap('plasma')
# convolution windows
fig, ax = plt.subplots(figsize=(10,5))
for mni in range(nb_motoneurons_full_pool):
    if mni == 0:
        plt.plot(twitch_convolution_window[mni]*twitch_force_motoneurons[mni]*1e-3, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 1, linewidth = 1.5, label = "smallest simulated motor unit")
    elif mni == nb_motoneurons_full_pool-1:
        plt.plot(twitch_convolution_window[mni]*twitch_force_motoneurons[mni]*1e-3, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 1, linewidth = 1.5, label = "largest simulated motor unit")
    else:
        plt.plot(twitch_convolution_window[mni]*twitch_force_motoneurons[mni]*1e-3, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5, linewidth = 2)
# Multiplying by "twitch_force_motoneurons*1e-3" to show the torque produced by a unique twitch (without summation)
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Torque (milliNewton/meter)")
plt.legend()
plt.title("Kernel for twitch torque convolution")
new_filename = f'Twitch_force_convolution_kernels.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [9]:
# Sanity check of the convolution/tetanus process to obtain force => fist create binary spike train of spike brusts with faster and faster frquencies to check the force response for a given frequency
frequency_of_bursts = 1 # burst every N seconds
duration_of_bursts = 0.5 # in seconds
max_pps_of_bursts = 100 # up to 50 pps
nb_of_bursts = np.round(max_pps_of_bursts*duration_of_bursts)
sanity_check_time = linspace(0, nb_of_bursts * frequency_of_bursts, int(np.round(fsamp * nb_of_bursts * frequency_of_bursts)))

firing_frequencies_MUtorque = []
firing_frequencies_MUtorque_corresponding_samples = []

binary_spike_test = np.zeros((1,len(sanity_check_time)))

for bursti in range(int(nb_of_bursts)):
    firing_frequencies_MUtorque.append(bursti*(1/duration_of_bursts))
    burst_temp = np.zeros(int(np.round(duration_of_bursts*fsamp)))
    burst_temp[np.round(linspace(0,(duration_of_bursts*fsamp)-1,
                bursti)).astype(int)] = 1
    corresponding_samples = np.arange( int(np.round(frequency_of_bursts * bursti * fsamp)), int(np.round(frequency_of_bursts * bursti * fsamp))+len(burst_temp) )
    firing_frequencies_MUtorque_corresponding_samples.append(corresponding_samples[0:len(burst_temp)])
    binary_spike_test[0,corresponding_samples] = burst_temp
    # binary_spike_test[0,corresponding_samples] = np.concatenate((burst_temp, np.zeros(len(burst_temp)) ))


In [ ]:
# Sanity check of the convolution/tetanus process to obtain force => visualize the response for the spike train created in the previous cell
test_summed_force = Convolve_to_get_force(binary_spike_test,[0],'absolute')
plt.figure(figsize=(100,40))
plt.subplot(211)
plt.plot(sanity_check_time, test_summed_force,label="force produced (smallest MN)",color=colormap_temp(0), linewidth=2, alpha = 1)
plt.hlines(motoneurons_tetanus_forces[0], 0, np.max(sanity_check_time), label = "Tetanus torque (limit)", color = "black", linestyles=":", linewidth = 5)
plt.hlines(motoneurons_tetanus_forces[0]*0.75, 0, np.max(sanity_check_time), label = "75% of tetanus torque", color = "red", linestyles=":", linewidth = 5)
plt.xlabel("Time (s)")
plt.ylabel("Torque produced (milliNetwon/meter)")
plt.title("Testing the twitch force summation up to the tetanus for the smallest MU")

plt.subplot(212)
test_summed_force = Convolve_to_get_force(binary_spike_test,[nb_motoneurons_full_pool-1],'absolute')
plt.plot(sanity_check_time, test_summed_force,label="force produced (largest MN)",color=colormap_temp(0.5), linewidth=2, alpha = 1)
plt.hlines(motoneurons_tetanus_forces[nb_motoneurons_full_pool-1], 0, np.max(sanity_check_time), label = "Tetanus torque (limit)", color = "black", linestyles=":", linewidth = 5)
plt.hlines(motoneurons_tetanus_forces[nb_motoneurons_full_pool-1]*0.75, 0, np.max(sanity_check_time), label = "75% of tetanus torque", color = "red", linestyles=":", linewidth = 5)
plt.xlabel("Time (s)")
plt.ylabel("Torque produced (milliNetwon/meter)")
plt.title("Testing the twitch force summation up to the tetanus for the largest MU")

max_force_temp = np.ceil(np.max(test_summed_force))
plt.subplot(211)
plt.ylim(0-(max_force_temp/10),max_force_temp+(max_force_temp/2))
plt.plot(sanity_check_time, binary_spike_test[0,:]*max_force_temp,label="spike train", color=[0.3,0.5,1], alpha = 0.25, linewidth = 3)
plt.xlim(0, np.max(sanity_check_time))
plt.legend()
plt.subplot(212)
plt.ylim(0-(max_force_temp/10),max_force_temp+(max_force_temp/2))
plt.plot(sanity_check_time, binary_spike_test[0,:]*max_force_temp,label="spike train", color=[0.3,0.5,1], alpha = 0.25, linewidth = 3)
plt.xlim(0, np.max(sanity_check_time))
plt.legend()

new_filename = f'Torque_reponse_smallest_largest_MUs_spike_bursts.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

In [ ]:
peak_force_per_MU_per_firing_freq = np.zeros((nb_motoneurons_full_pool,len(firing_frequencies_MUtorque)))
firing_freq_to_reach_tetanus = np.full(nb_motoneurons_full_pool, np.nan)
# mean_force_per_MU_per_firing_freq = np.zeros((nb_motoneurons_full_pool,len(firing_frequencies_MUtorque))) # over a 0.5s window when duration_of_bursts = 0.5

for mni in range(nb_motoneurons_full_pool):
    test_summed_force = Convolve_to_get_force(binary_spike_test,[mni],'absolute')

    for freqi in range(len(firing_frequencies_MUtorque)):
        temp_samples = firing_frequencies_MUtorque_corresponding_samples[freqi]
        if (freqi+1) < len(firing_frequencies_MUtorque): # only if not the last frequency (otherwise it will extend beyond the spike-train duration)
            temp_samples = np.concatenate((temp_samples, np.arange(len(twitch_convolution_window[mni])) + np.max(temp_samples) )) # extending the considered samples to account for the last spike (meaningful for low firing frequencies)
        peak_force_per_MU_per_firing_freq[mni,freqi] = np.max(test_summed_force[temp_samples])
        if isnan(firing_freq_to_reach_tetanus[mni]) and (peak_force_per_MU_per_firing_freq[mni,freqi] > 0.75 * motoneurons_tetanus_forces[mni]) : # if reaching at least 75% of max force (tetanic force)
            firing_freq_to_reach_tetanus[mni] = freqi
        # mean_force_per_MU_per_firing_freq[mni,freqi] = np.mean(test_summed_force[temp_samples])*(1/duration_of_bursts) # normalize for the "burst window"

plt.figure(figsize=(15,10))
for mni in range(nb_motoneurons_full_pool):
    plt.plot(firing_frequencies_MUtorque, peak_force_per_MU_per_firing_freq[mni,:], color=colormap_temp(mni/nb_motoneurons_full_pool), alpha = 0.5)
plt.xlabel("Firing frequency (pps)")
plt.ylabel("Torque (milliNewton/meter)")
plt.title("Peak torque of MU according to firing frequency (dark = small MU; light = large MU)")
new_filename = f'MU_percentage_of_peak_torque_relative_to_DR.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


# plt.figure(figsize=(20,15))
# for mni in range(nb_motoneurons_full_pool):
#     plt.plot(firing_frequencies_MUtorque, mean_force_per_MU_per_firing_freq[mni,:], color=colormap_temp(mni/nb_motoneurons_full_pool), alpha = 0.5)
# plt.xlabel("Firing frequency (pps)")
# plt.ylabel("Torque (milliNewton/meter)")
# plt.title(f"Mean torque produced over 1s (Y) at a given firing frequency (X) \n (dark = small MU; light = large MU)")

plt.figure(figsize=(10,10))
plt.subplot(211)
plt.plot(firing_freq_to_reach_tetanus, color='red', linewidth=3)
# plt.hlines(np.nanmean(firing_freq_to_reach_tetanus), xlim()[0], xlim()[1], color='red', linewidth = 5, alpha = 0.5, label = "mean")
plt.xlabel("MN index")
plt.ylabel("Firing frequency (pps)")
plt.ylim(0,max_pps_of_bursts)
plt.title("Firing frequency necessary to reach 75% of tetanus torque ~ MN index")
plt.subplot(212)
plt.plot(motoneuron_soma_diameters, firing_freq_to_reach_tetanus, color=[0.8,0,0], linewidth=3)
# plt.hlines(np.nanmean(firing_freq_to_reach_tetanus), xlim()[0], xlim()[1], color=[0.8,0,0], linewidth = 5, alpha = 0.5, label = "mean")
plt.xlabel("Soma diameter")
plt.ylabel("Firing frequency (pps)")
plt.ylim(0,max_pps_of_bursts)
plt.title("Firing frequency necessary to reach 75% of tetanus torque ~ MN size")
new_filename = f'MU_DR_needed_to_reach_peak_torque.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [12]:
# Define a low-pass filter
def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a
def lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Low-pass filter artifact removal
duration_to_remove = 1/window_beginning_ignore # in second
Wind_s = duration_to_remove * 2
HanningW = windows.hann(round(fsamp * Wind_s))
HanningW = HanningW[:int(np.round(len(HanningW)/2))]
nb_samples_artifact_removal_window = len(HanningW)

In [13]:
# When rounding 2.7 for example, 70% to get 3 and 30% chance to get 2
def probabilistic_round(number):
    lower = int(number)  # The lower integer
    upper = lower + 1    # The upper integer
    decimal_part = number - lower
    
    return upper if random.random() < decimal_part else lower

In [14]:
# GET SPIKE TRAINS AND BINARY SPIKE TRAINS
def Get_binary_spike_trains(spike_monitor, sim_duration):
    # Define time bins
    time_bins = np.arange(0, int(np.round((sim_duration*fsamp)))*second) * ms

    # Retrieve spikes and get binary spike trains
    spike_trains = []
    for mni in range(nb_motoneurons_full_pool):
        spike_trains.append(spike_monitor.spike_trains()[mni])
    
    # Initialize the binary spike train array
    binary_spike_trains = {}
    binary_spike_trains = np.zeros((nb_motoneurons_full_pool, len(time_bins)))
    # Convert spike times to binary spike train
    for neuron_idx in range(nb_motoneurons_full_pool):
        spikes = spike_trains[neuron_idx]
        spike_indices = np.searchsorted(time_bins, spikes)
        binary_spike_trains[neuron_idx, spike_indices-1] = 1 #-1 because of offset due to 0-indexing
    
    return spike_trains, binary_spike_trains

In [ ]:
# excitation_weight_smallest_MN = 0.5
# excitation_weight_largest_MN = 2
# excitation_weight_relationship_from_smallest_to_largest = 3
# # ^ just for testing purpose

# CREATE EXCITATORY INPUT WEIGHTS
# Necessary at this stage already, because of the MVC simulation
motoneurons_excitation_weights = np.ones(nb_motoneurons_full_pool)
for mni in range(nb_motoneurons_full_pool):
    motoneurons_excitation_weights[mni] = lerp(excitation_weight_smallest_MN, excitation_weight_largest_MN, motoneuron_normalized_soma_diameters[mni]**excitation_weight_relationship_from_smallest_to_largest)

plt.figure()
plt.plot(motoneurons_excitation_weights, color=[1,0.7,0.2], linewidth = 5)
plt.xlabel("MN index")
plt.ylabel("Weight of excitatory input")
plt.title("Weight of excitatory input ~ MN index")
new_filename = f'MWeights_excitatory_input_curve_MN_idx.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

plt.figure()
plt.plot(motoneuron_soma_diameters, motoneurons_excitation_weights, color=[1,0.7,0.2], linewidth = 5)
plt.xlabel("MN soma diameter")
plt.ylabel("Weight of excitatory input")
plt.title("Weight of excitatory input ~ MN size")
new_filename = f'MWeights_excitatory_input_curve_MN_size.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

In [ ]:
# Get max tetanic force of the simulated pool => by sending a very very high input to all MNs and recording the resulting force
# The force comes from the estimated peak torque values of individual motor units in the human tibialis anterior, so the low MVC values are not so unrealistic

# excit_input_for_MVC = 1000*1e-2 # for testing purposes
MVC_plateau_duration = 5 # in second
ramp_of_MVC_duration = 10 # in second
total_MVC_durarion = (MVC_plateau_duration + ramp_of_MVC_duration + 1) # +1 because one second of no input to get rid of artifacts

# Initialize inputs for MVC
# Generate empty input first (necessary to get rid of artifacts)
excit_input_with_ramp_MVC = np.zeros(int(np.round(1*fsamp)))
excit_input_with_ramp_MVC = np.append(excit_input_with_ramp_MVC,
    linspace(0,1,int(np.round(ramp_of_MVC_duration*fsamp))) * excit_input_for_MVC) # Generate ramp first
excit_input_with_ramp_MVC = np.append(excit_input_with_ramp_MVC,
                                      np.ones(int(np.round(MVC_plateau_duration*fsamp))) * excit_input_for_MVC) # then plateau

excit_input_per_MN = np.ones(( nb_motoneurons_full_pool, int(np.round(total_MVC_durarion*fsamp)) )) * excit_input_with_ramp_MVC
input_inhib_per_MN = np.zeros(( nb_motoneurons_full_pool,int(np.round(total_MVC_durarion*fsamp)) ))
for mni in range(nb_motoneurons_full_pool):
    excit_input_per_MN[mni,:] = excit_input_per_MN[mni,:] * motoneurons_excitation_weights[mni]
    excit_input_per_MN[mni,:] = np.clip(excit_input_per_MN[mni,:], a_min=0, a_max=None)# Clamp to 0 to avoid negative conductance
excit_input_per_MN = np.transpose(excit_input_per_MN)
input_excit = TimedArray(excit_input_per_MN * msiemens, dt=1*ms)
input_inhib_per_MN = np.transpose(input_inhib_per_MN)
input_inhib = TimedArray(input_inhib_per_MN * msiemens, dt=1*ms)

plt.figure()
plt.plot(excit_input_with_ramp_MVC, label="MVC excitatory input (ramp and plateau)", color = "C1")
plt.plot(input_inhib_per_MN[:,0], label="MVC inhibitory input (ramp and plateau)", color = "C4")
plt.legend()
plt.title("Input for MVC")

In [17]:
# RUN MVC SIMULATION

# Reset simulation
start_scope() # Re-initialize Brian
eqs_motoneuron = LIF_equations

# Groups of neurons
motoneurons = NeuronGroup(nb_motoneurons_full_pool, eqs_motoneuron,
                        threshold='v>voltage_thresh',
                        reset='v=voltage_rest',
                        refractory='refractory_period',
                        method=sim_method)
                        
# Initialize values
motoneurons.v = voltage_rest + (rand(nb_motoneurons_full_pool)*voltage_thresh) # in mV # uniform distribution between 0 and voltage threshold => prevents early synchronization
motoneurons.g_leak = motoneurons_membrane_conductance * msiemens # in milisiemens
motoneurons.C_m = motoneuron_capacitances * ufarad # in microfarads
motoneurons.I_th = motoneurons_rheobases * rheobase_scaling * nA # in nanoAmperes
motoneurons.refractory_period = motoneurons_refractory_periods * ms  # in milliseconds
motoneurons.input_weight = motoneuron_input_weights # dimensionless unit
# Monitors
monitor_state_motoneurons = StateMonitor(motoneurons, variables=True, record=True)
monitor_spikes_motoneurons = SpikeMonitor(motoneurons, record=True)

# Run simulation
run(total_MVC_durarion * second)


In [ ]:
# GET RESULTS OF MVC SIMULATION

# Get force only on the plateau
MVC_sim_samples = np.arange(0,int(np.round(total_MVC_durarion * fsamp)))
MVC_sim_samples = MVC_sim_samples[(ramp_of_MVC_duration+1)*fsamp:] # remove samples from the ramp
# Get spike trains
spike_trains, binary_spike_trains = Get_binary_spike_trains(monitor_spikes_motoneurons, total_MVC_durarion)
# Get force
MVC_force_absolute = Convolve_to_get_force(binary_spike_trains,np.arange(nb_motoneurons_full_pool),'absolute')
max_MVC_force_absolute = np.mean(MVC_force_absolute[MVC_sim_samples]) # only during the plateau

# Force per MU
force_total = zeros(len(binary_spike_trains[0,:]))
fig, ax = plt.subplots(figsize=(10, 5))
# Getting a smooth color blend from a given colormap
colormap_temp = cm.get_cmap('plasma')
for mni in range(nb_motoneurons_full_pool):
    temp_force = Convolve_to_get_force(np.reshape(binary_spike_trains[mni,:], (1, len(binary_spike_trains[mni,:]))), [mni], 'absolute') / 1000
    force_total += temp_force
    ax.plot(force_total, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5)
ax.plot(force_total, color='black', alpha = 1, linewidth = 2)
plt.title(f"Reconstructed torque (convolving spike train with twitch torque kernel)")
plt.ylabel("Torque (Newton/meter)")
plt.xlabel("Time (ms)")
plt.vlines((ramp_of_MVC_duration+1)*fsamp, ymin=0, ymax = ylim()[1], color="red", linewidth=2, alpha = 0.5)
new_filename = f'Max_force_for_simulated_MNs.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)


In [ ]:
# Get recruitment of MUs (clean version without noise, and for all MNs since the MVC ramp recruits all of them)
fig, ax1 = plt.subplots(figsize=(10,7))
ax1.plot(excit_input_with_ramp_MVC[:int(np.round((ramp_of_MVC_duration+1)*fsamp))], label="MVC excitatory input (ramp)", color = "C1",
         linewidth = 3)
MN_recruitment_thresholds_by_excitatory_input_clean_MVC = np.full(nb_motoneurons_full_pool, np.nan) # in milliSiemens of excitatory input signal
MN_recruitment_thresholds_by_excitatory_input_ratio = np.full(nb_motoneurons_full_pool, np.nan) # in milliSiemens of excitatory input signal
for mni in range(nb_motoneurons_full_pool):
    # RT as input during the median time of the first 3 firings
    temp_RT = (spike_trains[mni]/second)*fsamp
    temp_RT = np.median(temp_RT[0:2])
    if np.isnan(temp_RT)==False:
        MN_recruitment_thresholds_by_excitatory_input_clean_MVC[mni] = excit_input_with_ramp_MVC[int(np.round(temp_RT))]
        MN_recruitment_thresholds_by_excitatory_input_ratio[mni] = MN_recruitment_thresholds_by_excitatory_input_clean_MVC[mni] / MN_recruitment_thresholds_by_excitatory_input_clean_MVC[0]
        ax1.scatter(int(np.round(temp_RT)), MN_recruitment_thresholds_by_excitatory_input_clean_MVC[mni],
                    color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5, s=50)
ax2 = ax1.twinx()
ax2.plot((force_total[:int(np.round((ramp_of_MVC_duration+1)*fsamp))]/(max_MVC_force_absolute/1000))*100, label="Normalized force", color = "black", linewidth=2, alpha=0.5)
ax1.set_xlabel("Time (ms)")
ax1.set_ylabel("Excitatory input (mS)", color = "C1")
ax1.spines['left'].set_color('C1')  # Y-axis on the left
ax1.tick_params(axis='y', colors='C1')  # Tick labels for y-axis
ax2.set_ylabel("Force (% MVC)", color = "grey")
ax2.spines['right'].set_color('grey')  # Y-axis on the left
ax2.tick_params(axis='y', colors='grey')  # Tick labels for y-axis
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.title("Recruitment of MUs during the MVC ramp (dark dots = small MU; light dots = large MU)")
new_filename = f'MVC_ramp_recruitment_thresholds.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

plt.figure()
plt.plot(motoneuron_soma_diameters, MN_recruitment_thresholds_by_excitatory_input_ratio)
plt.xlabel("Soma diameter (micrometers)")
plt.ylabel("Recruitment threshold relative to smallest MN")
plt.title(f"Recruitment threshold ratio in escitatory input (relative to smallest MN) \n Chosen excitatoy input bias = {excitation_weight_smallest_MN} for smallest MN; {excitation_weight_largest_MN} for largest MN \n RT range = {np.round(np.max(MN_recruitment_thresholds_by_excitatory_input_ratio)*100)/100}-fold")
new_filename = f'MVC_ramp_recruitment_thresholds_ratio.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

In [ ]:
### Get discharge characteristics of motoneurons on the MVC plateau
spike_trains = [array[array >= (ramp_of_MVC_duration+1)*second] for array in spike_trains] # removing spikes that belong to the ramp in "spike_trains"
mean_firing_rate = {}
std_firing_rate = {}
fig, axs = plt.subplots()
firing_rates = []
highest_ISIs = []
for mni in range(nb_motoneurons_full_pool):
    if len(spike_trains[mni]) <= 1:
        highest_ISIs.append(MVC_plateau_duration*fsamp)
    else:
        highest_ISIs.append(max(diff(spike_trains[mni])))
    # firing_rate_temp = len(spike_trains[mni]) / MVC_plateau_duration
    if len(spike_trains[mni]) > 1:
        firing_rate_temp = 1/np.mean(np.diff(spike_trains[mni])) 
    else:
        firing_rate_temp = 0
    firing_rates.append(firing_rate_temp)
# Convert to a numpy array for easier calculations
firing_rates = np.array(firing_rates)
print(f"Number of silent MUs during the MVC = {np.sum(firing_rates<1)} (there should be none)")
# Calculate mean and standard deviation of the firing rates
mean_firing_rate = np.mean(firing_rates)
std_firing_rate = np.std(firing_rates)
# Motoneurons' firing rates results
axs.hist(firing_rates, edgecolor='white', alpha=1, color='C1')
axs.axvline(x = mean_firing_rate, linestyle='--', linewidth=2, label='Mean firing rate', color='red')
axs.set_xlabel("Mean firing rate (pps)")
axs.set_ylabel("Motoneuron count count")
plt.tight_layout(rect=[0,0,1,0.96])
plt.suptitle("Histogram of motoneurons' firing rate - high excitatory input simulation (for MVC estimation)")
new_filename = f'MVC_Hist_Discharge_rates_MVC.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

plt.figure(figsize=(10,10))
plt.subplot(211)
plt.plot(firing_rates, color = 'C1', linewidth = 3)
plt.ylabel("Firing rate (pps)")
plt.xlabel("MN index")
plt.title("Firing rates during MVC ~ MN index")
plt.subplot(212)
plt.plot(motoneuron_soma_diameters, firing_rates, color = 'C1', linewidth = 3)
plt.ylabel("Firing rate (pps)")
plt.xlabel("soma size (micrometers)")
plt.title("Firing rates during MVC ~ soma diameter")
new_filename = f'MVC_Discharge_rates.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

print(f'Estimated MVC (Newton/meter) = {np.round(max_MVC_force_absolute)/1000} (for {nb_motoneurons_full_pool} motor units)')  # /1000 so that it is expressed in N/m
print(f'MVC (Newton/meter) would be {(np.round(max_MVC_force_absolute)/1000) / (nb_motoneurons_full_pool/300)} with 300 motor units')
target_force_level_absolute_value = max_MVC_force_absolute * (target_force_level/100)  # /100 because 'target_force_level' is expressed as a percentage
print(f'Target torque level (Netwon/meter) = {np.round(target_force_level_absolute_value)/1000}')  # /1000 so that it is expressed in N/m

In [ ]:
# Define force target
if (target_type == 'plateau'):
    print("Plateau force target")
    target_force = ones(int(duration_with_ignored_window * fsamp))*target_force_level
elif (target_type == 'sinusoid'):
    print("Sinusoidal force target")
    target_force = ones(int(duration_with_ignored_window * fsamp))*target_force_level
    sinusoids_temp =  np.linspace(0, duration_with_ignored_window, int(duration_with_ignored_window*fsamp), endpoint=False)
    sinusoids_temp = sinusoids_temp / second
    sinusoids_temp = (target_force * 0.25) * np.sin(2 * np.pi * target_force_sin_freq * sinusoids_temp)
    target_force = target_force + sinusoids_temp
elif (target_type == 'trapezoid'):
    print("Trapezoidal force target")
    target_force = np.zeros(int(window_beginning_ignore*fsamp))
    ramp_up = linspace(0,target_force_level,ramp_duration*fsamp)
    target_force = np.append(target_force,ramp_up)
    plateau = ones(int(plateau_duration * fsamp))*target_force_level
    target_force = np.append(target_force,plateau)
    ramp_down = linspace(target_force_level,0,ramp_duration*fsamp)
    target_force = np.append(target_force,ramp_down)
    target_force = np.append(target_force,np.zeros(int(window_end_ignore*fsamp)))
else:
    print("Please select a valid target type type")
    sys.exit()

plt.plot(target_force, color='black', alpha = 0.5, linewidth = 2)
plt.xlabel("Time (ms)")
plt.ylabel("Target force (% MVC)")
plt.title("Target force")
new_filename = f'Target_force.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [22]:
time = linspace(0,duration_with_ignored_window/second,int(duration_with_ignored_window/second*fsamp))

In [ ]:
# INHIBITORY INPUT
Wind_s = 1/low_pass_filter_of_inhibitory_input  # hanning window duration.
HanningW = 2 / round(fsamp * Wind_s) * windows.hann(round(fsamp * Wind_s))  # unitary area

# Create inhibitory input
inhib_input = {}
if inhibitory_input_source == 'generate_synthetic_input':
    for inhibiti in range(nb_inhibitory_input):
        inhib_input[inhibiti] = []
        temp_inhib = np.random.normal(0, 1, int(duration_with_ignored_window * fsamp))
        temp_inhib[int(duration_with_ignored_window * fsamp)-int(np.round(window_beginning_ignore/1)):int(duration_with_ignored_window * fsamp)] = 0
        temp_inhib[0:int(np.round(window_beginning_ignore/1))] = 0
        temp_inhib = lowpass_filter(temp_inhib, low_pass_filter_of_inhibitory_input, fsamp)
        # temp_inhib = filtfilt(HanningW, 1, temp_inhib * fsamp)
        temp_inhib = temp_inhib - np.mean(temp_inhib)
        temp_inhib = temp_inhib / np.std(temp_inhib)
        temp_inhib = temp_inhib * inhibitory_input_std
        temp_inhib = temp_inhib + inhibitory_input_mean
        inhib_input[inhibiti].append(temp_inhib)
elif inhibitory_input_source == 'load_synthetic_input':
    synthetic_signals_dataframe = pd.read_csv(inhibitory_input_sourcefile)
    if inhibitory_input_sourcefile_fsamp != fsamp:
        from scipy.interpolate import interp1d
        # Calculate the time array for the original signal
        loaded_signal_time = np.arange(len(synthetic_signals_dataframe)) / inhibitory_input_sourcefile_fsamp
        # Calculate the number of samples in the resampled signal
        number_of_samples = int(len(synthetic_signals_dataframe) * fsamp / inhibitory_input_sourcefile_fsamp)
        # Calculate the time array for the resampled signal
        resampled_time = np.linspace(loaded_signal_time[0], loaded_signal_time[-1], number_of_samples)
    for inhibiti in range(nb_inhibitory_input):
        inhib_input[inhibiti] = []
        temp_inhib = synthetic_signals_dataframe[f'{inhibiti}'].values
        if inhibitory_input_sourcefile_fsamp != fsamp:
            # Create an interpolation function
            resample_loaded_signal_function = interp1d(loaded_signal_time, temp_inhib, kind='linear')
            temp_inhib = resample_loaded_signal_function(resampled_time)
        temp_inhib = temp_inhib[:len(time)] # cut the signal for it to be the right size
        temp_inhib = temp_inhib * inhibitory_input_std
        temp_inhib = temp_inhib + inhibitory_input_mean
        inhib_input[inhibiti].append(temp_inhib)

plt.figure()
for inhibiti in range(nb_inhibitory_input):
    plt.plot(np.transpose(inhib_input[inhibiti]), alpha = 0.5, label=f'Inhibitory input #{inhibiti}', color = 'C4')
plt.xlabel("Time (ms)")
plt.ylabel("Inhibitory input(s)")
plt.legend()
new_filename = f'Inhibitory_input_signal.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [58]:
inhibition_weight_distribution = 'mixed_additive' #'multimodal' 'exponential' 'mixed_additive' 'mixed_multiplicative'
# 'multimodal' = MN will receive inhibition according the weights distributed according to a normal distribution. For example, for 50-50 distribution of either 0 or 1, use...
#   # inhibition_multimodal_number_of_modes = 2
#   # inhibition_multimodal_weights_distrib_means = [0, 1]
#   # inhibition_multimodal_weights_distrib_stds = [0, 0]
inhibition_multimodal_number_of_modes = 1 # used only if 'inhibition_weight_distribution = multimodal'
inhibition_multimodal_weights_distrib_means = [0] # os.getenv('inhibition_multimodal_weights_distrib_means') # [0.1, 0.9] # the number of element should be = to 'inhibition_multimodal_number_of_modes' # used only if 'inhibition_weight_distribution = multimodal'
    # inhibition_multimodal_weights_distrib_means = eval(inhibition_multimodal_weights_distrib_means)
inhibition_multimodal_weights_distrib_stds = [0.1] # os.getenv('inhibition_multimodal_weights_distrib_stds') # [0.02, 0.02] # the number of element should be = to 'inhibition_multimodal_number_of_modes' # used only if 'inhibition_weight_distribution = multimodal'
    # inhibition_multimodal_weights_distrib_stds = eval(inhibition_multimodal_weights_distrib_stds)
inhibition_multimodal_weights_proportions = [1] # os.getenv('inhibition_multimodal_weights_proportions') # [0.5, 0.5]
    # inhibition_multimodal_weights_proportions = eval(inhibition_multimodal_weights_proportions)
# the number of elements should be equal to 'inhibition_multimodal_number_of_modes'. If the proportions add up to more than 1, an error will occur. If they sum to less than 1, the remaining MUs not in any group will be assigned a weight of 0. # used only if 'inhibition_weight_distribution = multimodal'
# 'exponential' = MNs receiving inhibition will receive increasing or decreasing amount of inhibition according to their sizes - Martinez Valdes J physiol 2020 model = https://physoc.onlinelibrary.wiley.com/doi/full/10.1113/JP279225 
inhibition_exponential_exponent_weights = 2 # used only if 'inhibition_weight_distribution = exponential'
inhibition_exponential_constant_weights = 1.5 # used only if 'inhibition_weight_distribution = exponential'
inhibition_exponential_offset_weights = 0 # 0 # used only if 'inhibition_weight_distribution = exponential'
#   # Distribution of inhibitory weights = constant * MN_size^(exponent) + offset

In [ ]:
# Distribute inhibitory input

if (inhibition_weight_distribution != 'multimodal') and (inhibition_weight_distribution != 'exponential') and (inhibition_weight_distribution != 'mixed_additive') and (inhibition_weight_distribution != 'mixed_multiplicative'):
    print("Please select a valid inhibition distribution type ('multimodal' or 'exponential' or 'mixed_additive' or 'mixed_multiplicative')")
    sys.exit()

if (inhibition_weight_distribution == 'exponential' or inhibition_weight_distribution == 'mixed_additive' or inhibition_weight_distribution == 'mixed_multiplicative'):
    MN_inhibition_weights_curve = np.zeros(nb_motoneurons_full_pool)
    for mni in range(nb_motoneurons_full_pool):
        MN_inhibition_weights_curve[mni] = inhibition_exponential_constant_weights * (1-motoneuron_normalized_soma_diameters[mni])**inhibition_exponential_exponent_weights + inhibition_exponential_offset_weights
    plt.figure()
    plt.plot(MN_inhibition_weights_curve, color='C4')
    plt.ylim([-0.1,np.max([1.1,inhibition_exponential_constant_weights+0.1])])
    plt.ylabel("Inhibitory input weights (only the size-inhibition relationship)")
    plt.xlabel("MN index")
    plt.title("Inhibition distribution curve according to index (only the size-inhibition relationship)")

    plt.figure()
    plt.scatter(motoneuron_normalized_soma_diameters,MN_inhibition_weights_curve, color='C4')
    plt.ylim([-0.1,np.max([1.1,inhibition_exponential_constant_weights+0.1])])
    plt.xlabel("Normalized MN size")
    plt.ylabel('Inhibitory input weights (only the size-inhibition relationship)')
    plt.title("Distribution of inhibitory inputs according to SIZE (only the size-inhibition relationship)")
    new_filename = f'Inhibitory_input_distrib_relative_to_size.png'
    save_file_path = os.path.join(new_directory, new_filename)
    plt.savefig(save_file_path)
    plt.show()

MN_inhibition_weights = {}
if nb_inhibitory_input == 0:
    MN_inhibition_weights[0] = np.zeros(nb_motoneurons_full_pool)
else:
    for inhibiti in range(nb_inhibitory_input):
        MN_inhibition_weights[inhibiti] = np.zeros(nb_motoneurons_full_pool)
        if (inhibition_weight_distribution != 'exponential'):
            # separate motor neurons into groups of equivalent sizes
            #   # Calculate the number of elements in each group
            group_sizes = [int(nb_motoneurons_full_pool * p) for p in inhibition_multimodal_weights_proportions]
            if sum(inhibition_multimodal_weights_proportions) > 1 :
                print("The proportion of motor neurons in the multimodal distribution sums to more than 1. Please ensure that the sum of proportions is <= 1")
                sys.exit()
            #   # Shuffle the indices of the motor neurons
            MN_idx_shuffled_temp = np.arange(nb_motoneurons_full_pool)
            np.random.shuffle(MN_idx_shuffled_temp)
            #   # Split the indices into the groups
            MN_idx_in_groups_for_multimodal_distribution = []
            start_idx = 0
            for size in group_sizes:
                MN_idx_in_groups_for_multimodal_distribution.append(MN_idx_shuffled_temp[start_idx:start_idx + size])
                start_idx += size
            for groupi in range(inhibition_multimodal_number_of_modes):
                # # generate the normal distrbution for group 'groupi'
                inhib_weights_for_current_group_temp = np.random.normal(inhibition_multimodal_weights_distrib_means[groupi],
                                                                inhibition_multimodal_weights_distrib_stds[groupi],
                                                                len(MN_idx_in_groups_for_multimodal_distribution[groupi]))
                iter_mn_temp = 0
                for mni in MN_idx_in_groups_for_multimodal_distribution[groupi]:
                    MN_inhibition_weights[inhibiti][mni] = inhib_weights_for_current_group_temp[iter_mn_temp]
                    iter_mn_temp += 1
            #   # If there are remaining elements, assign a weight of zero
            remaining_elements = MN_idx_shuffled_temp[start_idx:]
            MN_inhibition_weights[inhibiti][remaining_elements] = 0
            if (inhibition_weight_distribution == 'mixed_additive') or (inhibition_weight_distribution == 'mixed_multiplicative'):
                for mni in range(nb_motoneurons_full_pool):
                    if (inhibition_weight_distribution == 'mixed_additive'):
                        MN_inhibition_weights[inhibiti][mni] = MN_inhibition_weights_curve[mni]+MN_inhibition_weights[inhibiti][mni]
                    elif (inhibition_weight_distribution == 'mixed_multiplicative'):
                        MN_inhibition_weights[inhibiti][mni] = MN_inhibition_weights_curve[mni]*MN_inhibition_weights[inhibiti][mni]

        else: # (inhibition_weight_distribution == 'exponential'):
            for mni in range(nb_motoneurons_full_pool):
                MN_inhibition_weights[inhibiti][mni] = MN_inhibition_weights_curve[mni]

# Clamp all weights between 0 and +inf to avoid negative weights
for inhibiti in range(nb_inhibitory_input):
    for mni in range(nb_motoneurons_full_pool):
        MN_inhibition_weights[inhibiti][mni] = np.clip(MN_inhibition_weights[inhibiti][mni],0,inf)

fig, ax = plt.subplots(1,1, figsize=(15,4))
x_plot_mns = range(nb_motoneurons_full_pool)
bottom_barplot = np.zeros(nb_motoneurons_full_pool)
for inhibiti in range(nb_inhibitory_input):
    vstack_inhibi_temp = MN_inhibition_weights[inhibiti]
    ax.bar(x_plot_mns,
        vstack_inhibi_temp,
        bottom = bottom_barplot,
        label = f"inhibitory input #{inhibiti}", color="C4")
    bottom_barplot += vstack_inhibi_temp
ax.set_xlabel('Motoneurons')
ax.set_ylabel('Inhibitory input weights')
# ax.set_ylim(0,1)
plt.suptitle("Distribution of inhibitory inputs according to INDICES (final)")
new_filename = f'Inhibitory_input_distrib_relative_to_indices.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

plt.figure(figsize=(15,4))
for inhibiti in range(nb_inhibitory_input):
    # plt.hist(MN_inhibition_weights[inhibiti], edgecolor='white', density=True, color='C4', alpha = 0.5)
    plt.hist(MN_inhibition_weights[inhibiti], edgecolor='white', color='C4', alpha = 0.5)
plt.title('Histogram of weights for inhibitory input distribution (final)')
plt.xlabel('Inhibition weight')
plt.ylabel('Count (nb of motor neurons)')
# plt.xlim(-0.1,1.1)
new_filename = f'Inhibitory_input_weight_distrib_hist.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()
    

In [ ]:
inhibitory_input_power_integral_0_5_hz = []
for inhibiti in range(nb_inhibitory_input):
    inhib_input_temp = np.copy(np.squeeze(np.array(inhib_input[inhibiti])))
    # remove artifacts of low pass filter
    inhib_input_temp[0:int(window_beginning_ignore*fsamp)] = inhibitory_input_mean
    inhib_input_temp[len(time)-int(window_end_ignore*fsamp):len(time)] = inhibitory_input_mean
    # Normalize
    inhib_input_temp = ((inhib_input_temp - np.mean(inhib_input_temp)) / np.std(inhib_input_temp)) * inhibitory_input_std
    N = len(inhib_input_temp)
    yf = fft(inhib_input_temp)
    xf = fftfreq(N, 1 / fsamp)
    power_spectrum_temp = (np.abs(yf[:N//2])**2) / N
    inhibitory_input_power_integral = np.sum(power_spectrum_temp)
    if inhibiti == 0:
        idx_corresponding_to_5hz = int(np.round((N/fsamp)*5))
    power_spectrum_temp_0_5hz = power_spectrum_temp[:idx_corresponding_to_5hz]
    inhibitory_input_power_integral_0_5_hz.append(np.sum(power_spectrum_temp_0_5hz))
    plt.plot(xf[:N//2], power_spectrum_temp, color = 'C4', alpha = 1/nb_inhibitory_input)
inhibitory_input_power_integral_0_5_hz_mean = np.mean(inhibitory_input_power_integral_0_5_hz)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.title("Power spectrum of the inhibitory inputs")
plt.xlim([0,20])
new_filename = f'Signal_inhibitory_input_power_spectrum.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

In [ ]:
# Generate independent input to MNs ######################

# Low-pass filter artifact removal
duration_to_remove = 1 # in second
Wind_s = duration_to_remove * 2
HanningW = windows.hann(round(fsamp * Wind_s))
HanningW = HanningW[:int(np.round(len(HanningW)/2))]
nb_samples_artifact_removal_window = len(HanningW)

plt.figure(figsize=(10,20))
independent_noise_excit = np.zeros((nb_motoneurons_full_pool,len(time)))
independent_noise_inhib = np.zeros((nb_motoneurons_full_pool,len(time)))
independent_excit_noise_power_integral_0_5_hz = []
independent_inhib_noise_power_integral_0_5_hz = []

for mni in range(nb_motoneurons_full_pool):
    # Excitatory noise
    independent_noise_excit[mni,:] = randn(len(time)) # random number with mean = 0 and std = 1
    independent_noise_excit[mni,:] = lowpass_filter(independent_noise_excit[mni,:], 50, fsamp, 3) # 3rd order 50-hz low pass filter
    # start of signal - artifact removal
    independent_noise_excit[mni,0:nb_samples_artifact_removal_window] = independent_noise_excit[mni,0:nb_samples_artifact_removal_window] * HanningW
    # end of signal - artifact removal
    independent_noise_excit[mni,len(independent_noise_excit[mni,:])-nb_samples_artifact_removal_window:len(independent_noise_excit[mni,:])] = independent_noise_excit[mni,len(independent_noise_excit[mni,:])-nb_samples_artifact_removal_window:len(independent_noise_excit[mni,:])] * HanningW[::-1]
    # Scale the input appropriately
    independent_noise_excit[mni,:] = (((independent_noise_excit[mni,:] - np.mean(independent_noise_excit[mni,:])) / np.std(independent_noise_excit[mni,:])) *
                                    independent_noise_ratio_std * excitatory_input_std * motoneurons_excitation_weights[mni]) + 0 # noise has a mean of 0 # * motoneurons_excitation_weights because scaling according to the weight of excitation received
    plt.subplot(411)
    plt.plot(time,independent_noise_excit[mni,:],color='C1',alpha=0.02)
    if mni == 0:
        plt.title("Independent noise of excitatory input (conductance fluctuation)")
        plt.xlabel("Time (s)")
        plt.ylabel("Noise amplitude (conductance fluctuations, in milisiemens)")
    # Power spectrum - excitatory input noise
    N = len(independent_noise_excit[mni,:])
    yf = fft(independent_noise_excit[mni,:])
    xf = fftfreq(N, 1 / fsamp)
    power_spectrum_temp = (np.abs(yf[:N//2])**2) / N
    independent_noise_power_integral = np.sum(power_spectrum_temp)
    if mni == 0:
        idx_corresponding_to_5hz = int(np.round((N/fsamp)*5))
    power_spectrum_temp_0_5hz = power_spectrum_temp[:idx_corresponding_to_5hz]
    independent_excit_noise_power_integral_0_5_hz.append(np.sum(power_spectrum_temp_0_5hz))
    plt.subplot(412)
    plt.plot(xf[:N//2], power_spectrum_temp, color = 'C1', alpha = 0.02)
    if mni == 0:
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Power")
        plt.title("Power spectrum of the excitatory noise inputs")
        plt.xlim([0,80])

    # Inhibitory noise
    independent_noise_inhib[mni,:] = randn(len(time)) # random number with mean = 0 and std = 1
    independent_noise_inhib[mni,:] = lowpass_filter(independent_noise_inhib[mni,:], 50, fsamp, 3) # 3rd order 50-hz low pass filter
    # start of signal - artifact removal
    independent_noise_inhib[mni,0:nb_samples_artifact_removal_window] = independent_noise_inhib[mni,0:nb_samples_artifact_removal_window] * HanningW
    # end of signal - artifact removal
    independent_noise_inhib[mni,len(independent_noise_inhib[mni,:])-nb_samples_artifact_removal_window:len(independent_noise_inhib[mni,:])] = independent_noise_inhib[mni,len(independent_noise_inhib[mni,:])-nb_samples_artifact_removal_window:len(independent_noise_inhib[mni,:])] * HanningW[::-1]
    # Scale the input appropriately
    independent_noise_inhib[mni,:] = (((independent_noise_inhib[mni,:] - np.mean(independent_noise_inhib[mni,:])) / np.std(independent_noise_inhib[mni,:])) * 
                                      independent_noise_ratio_std * inhibitory_input_std * MN_inhibition_weights[0][mni]) + 0 # noise has a mean of 0 # * MN_inhibition_weights[0] because scaling according to the weight of inhibition received (considering only the 1st inhibitory input)
    plt.subplot(413)
    plt.plot(time,independent_noise_inhib[mni],color='C4',alpha=0.02)
    if mni == 0:
        plt.title("Independent noise of inhibitory input (conductance fluctuation)")
        plt.xlabel("Time (s)")
        plt.ylabel("Noise amplitude (conductance fluctuations, in milisiemens)")
    # Power spectrum - inhibitory input noise
    N = len(independent_noise_inhib[mni,:])
    yf = fft(independent_noise_inhib[mni,:])
    xf = fftfreq(N, 1 / fsamp)
    power_spectrum_temp = (np.abs(yf[:N//2])**2) / N
    independent_noise_power_integral = np.sum(power_spectrum_temp)
    # if mni == 0:
    #     idx_corresponding_to_5hz = int(np.round((N/fsamp)*5))
    power_spectrum_temp_0_5hz = power_spectrum_temp[:idx_corresponding_to_5hz]
    independent_inhib_noise_power_integral_0_5_hz.append(np.sum(power_spectrum_temp_0_5hz))
    plt.subplot(414)
    plt.plot(xf[:N//2], power_spectrum_temp, color = 'C4', alpha = 0.02)
    if mni == 0:
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Power")
        plt.title("Power spectrum of the inhibitory noise inputs")
        plt.xlim([0,80])

independent_excit_noise_power_integral_0_5_hz_mean = np.mean(independent_excit_noise_power_integral_0_5_hz)
independent_inhib_noise_power_integral_0_5_hz_mean = np.mean(independent_inhib_noise_power_integral_0_5_hz)
new_filename = f'Signal_independent_noise.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
# plt.show() # Takes a long time to display everything as a plot (tens of seconds)

In [ ]:
# Create the inhibitory signals - including the independent noise of each MN (the inhibitory signal is not learned/otpimized, so it can be created now)
# input_excit and input_inhib should be clamped to 0 to avoid negative conductances

inhib_input_per_MN = np.zeros((nb_motoneurons_full_pool,len(time)))
for mni in range(nb_motoneurons_full_pool):
    inhib_input_per_MN[mni,:] = np.zeros(len(time))
    for inhibiti in range(nb_inhibitory_input):
        inhib_input_per_MN[mni,:] += inhib_input[inhibiti][0] * MN_inhibition_weights[inhibiti][mni]
        inhib_input_per_MN[mni,:] += independent_noise_inhib[mni,:] * MN_inhibition_weights[inhibiti][mni]
        inhib_input_per_MN[mni,:] = np.clip(inhib_input_per_MN[mni,:], a_min=0, a_max=None)# Clamp to 0 to avoid negative conductance
inhib_input_per_MN = np.transpose(inhib_input_per_MN)


plt.figure(figsize=(10,5))
for mni in range(nb_motoneurons_full_pool):
    plt.plot(time,inhib_input_per_MN[:,mni],color='C4',alpha=0.05)
    plt.xlabel("Time (s)")
    plt.ylabel("Inhibition amplitude")
    plt.title("Inhibition of each MN") 

In [ ]:
# Initialize excitatory input current + Store initial excitatory input
max_baseline_excit = excitatory_input_baseline
excit_input_all_MNs = (target_force / target_force_level) * excitatory_input_baseline
excit_input_all_MNs = (excit_input_all_MNs / np.max(excit_input_all_MNs)) * max_baseline_excit # normalize to 1, and multiply by baseline so that the baseline is the max value
initial_excit_signal = excit_input_all_MNs.copy()
samples_of_interest = list(range((window_beginning_ignore*fsamp),len(time)-(window_end_ignore*fsamp)))

# Inhibitory input to all MNs has already been created in the previous cell

plt.figure(figsize=(10,5))
plt.plot(initial_excit_signal)
plt.xlabel("Time (samples)")
plt.ylabel("Amplitude of excitatory input (millisiemens)")
plt.title("Baseline (before optimisation) excitatory signal")

In [149]:
# Just for testing purposes
# max_num_optimization_iterations = 5
# adam_learning_rate = 0.1
# excitatory_input_baseline = 150*1e-2

In [ ]:
### OPTIMIZE INPUT FOR FORCE TARGET - iterate simulations

# Initialize inhibitory signal to be distributed
if (keep_force_constant_despite_inhib == True):
    input_inhib = TimedArray(inhib_input_per_MN * msiemens, dt=1*ms)
else: # else ignore inhibition for the input optimization process
    input_inhib = TimedArray(np.zeros((len(time),nb_motoneurons_full_pool)) * msiemens, dt=1*ms)

# Define the cost function
def cost_function(output_force_var, target_force_var):
    return np.sum(abs(output_force_var - target_force_var))

# Gradient descent parameters
learning_rate = adam_learning_rate

# Adam optimizer parameters
initial_alpha = learning_rate  # Initial learning rate
beta1 = 0.1 #0.9
beta2 = 0.5 #0.999
epsilon = 1e-8
m = np.zeros_like(excit_input_all_MNs)
v = np.zeros_like(excit_input_all_MNs)
timestep = 0

# Lists to store error and output force for plotting
errors = []
initial_output_force = None

# Reset simulation
start_scope() # Re-initialize the simulation

optimized_excit_signal = np.copy(initial_excit_signal)
best_cost = np.Inf
best_iter_idx = 0

# plot that will be updated at each iteration
colormap_temp = cm.get_cmap('viridis')
plt.figure(figsize=(15,12))
plt.subplot(211)
plt.plot(target_force, color = 'black', linestyle = ':', linewidth = 3)
plt.subplot(212)
plt.plot(optimized_excit_signal, color = 'blue', linewidth = 2, alpha = 0.7, label = "Initial input signal")

# Optimization loop using Adam optimizer
for iteration in range(max_num_optimization_iterations):
    timestep += 1

    # Low-pass filter the current excitatory input signal to prevent the "learning/optimization" towards an oscillating common input
    # 1 hz cut-off
    excit_input_all_MNs = lowpass_filter(excit_input_all_MNs, 1, fsamp)
    # remove artifacts of low pass filter
    excit_input_all_MNs[0:int(np.round(window_beginning_ignore*fsamp)*0.5)] = 0
    excit_input_all_MNs[len(time)-int(np.round((window_end_ignore*fsamp)*0.5)):len(time)] = 0
    # Prevent common excitatory input from going negative (it happens at the beginning of slopes)
    excit_input_all_MNs[excit_input_all_MNs < 0] = 0

    # Update input current in the neuron group
    excit_input_per_MN = np.zeros((nb_motoneurons_full_pool,len(time)))
    for mni in range(nb_motoneurons_full_pool):
        excit_input_per_MN[mni,:] = excit_input_all_MNs * motoneurons_excitation_weights[mni]
        excit_input_per_MN[mni,:] += independent_noise_excit[mni,:]
        excit_input_per_MN[mni,:] = np.clip(excit_input_per_MN[mni,:], a_min=0, a_max=None)# Clamp to 0 to avoid negative conductance
    excit_input_per_MN = np.transpose(excit_input_per_MN)
    input_excit = TimedArray(excit_input_per_MN * msiemens, dt=1*ms)

    eqs_motoneuron = LIF_equations

    # Groups of neurons
    motoneurons = NeuronGroup(nb_motoneurons_full_pool, eqs_motoneuron,
                            threshold='v>voltage_thresh',
                            reset='v=voltage_rest',
                            refractory='refractory_period',
                            method=sim_method)
                            
    # Initialize values
    motoneurons.v = voltage_rest # in mV #
    motoneurons.g_leak = motoneurons_membrane_conductance * msiemens # in milisiemens
    motoneurons.C_m = motoneuron_capacitances * ufarad # in microfarads
    motoneurons.I_th = motoneurons_rheobases * rheobase_scaling * nA # in nanoAmperes
    motoneurons.refractory_period = motoneurons_refractory_periods * ms  # in milliseconds
    motoneurons.input_weight = motoneuron_input_weights # dimensionless unit
    # Monitors
    monitor_spikes_motoneurons = SpikeMonitor(motoneurons, record=True)

    # Run simulation
    run(duration_with_ignored_window)

    # Get spike trains
    spike_trains, binary_spike_trains = Get_binary_spike_trains(monitor_spikes_motoneurons, duration_with_ignored_window)

    # Get force
    output_force = Convolve_to_get_force(binary_spike_trains,np.arange(nb_motoneurons_full_pool),'normalized')
    # if low_pass_filter_force:
        # output_force = lowpass_filter(output_force, low_pass_filter_of_force_cutoff, fsamp)
    if iteration == 0:
        initial_output_force = output_force
    # During the learning phase, low-pass filtering the force helps the algorithm converge towards a better common input "solution"
    output_force = lowpass_filter(output_force, 1, fsamp)

    # Calculate cost
    # output_force_windowed = output_force[samples_of_interest]
    # target_force_windowed = target_force[samples_of_interest]
    output_force_windowed = output_force[window_beginning_ignore*fsamp:len(output_force)-(window_end_ignore*fsamp)]
    target_force_windowed = target_force[window_beginning_ignore*fsamp:len(output_force)-(window_end_ignore*fsamp)]
    cost = cost_function(output_force_windowed, target_force_windowed)/len(samples_of_interest)
    if consider_only_plateau_for_cost_optimization and (target_type == 'trapezoid'):
        output_force_windowed = output_force[(window_beginning_ignore+ramp_duration)*fsamp:len(output_force)-((window_end_ignore+ramp_duration)*fsamp)]
        target_force_windowed = target_force[(window_beginning_ignore+ramp_duration)*fsamp:len(output_force)-((window_end_ignore+ramp_duration)*fsamp)]
        cost_only_plateau = cost_function(output_force_windowed, target_force_windowed)/len(samples_of_interest)
        print(f'Iteration {iteration + 1}, Cost (only on plateau): {np.round(cost_only_plateau*100)/100} (mean error in % of MVC)')
    errors.append(cost)
    print(f'Iteration {iteration + 1}, Cost: {np.round(cost*100)/100} (mean error in % of MVC)')

    if cost < best_cost:
        print(f'New best control signal')
        best_output_force = np.copy(output_force)
        best_cost = cost
        optimized_excit_signal = np.copy(excit_input_all_MNs)
        best_iter_idx = iteration

    # Adjust learning rate inversely proportional to the error
    # alpha = min(cost*learning_rate*0.5,learning_rate)
    alpha = learning_rate
    
    # Compute gradients
    gradients = (output_force - target_force)

    # Adam optimizer updates
    m = beta1 * m + (1 - beta1) * gradients
    v = beta2 * v + (1 - beta2) * (gradients ** 2)
    m_hat = m / (1 - beta1 ** timestep)
    v_hat = v / (1 - beta2 ** timestep)
    excit_input_all_MNs -= alpha * m_hat / (np.sqrt(v_hat) + epsilon)

    # Update figure
    plt.subplot(211)
    plt.plot(output_force, color=colormap_temp(iteration/max_num_optimization_iterations), linewidth=2, alpha = 0.7)
    plt.subplot(212)
    plt.plot(excit_input_all_MNs, color = colormap_temp(iteration/max_num_optimization_iterations), linewidth = 2, alpha = 0.7)

    if consider_only_plateau_for_cost_optimization and (target_type == 'trapezoid'):
        if cost_only_plateau < stop_optimizing_if_mean_error_is_below:
            print("Stopping optimization because the cost (% of MVC error) has reached a satisfying level on the plateau part of the simulated contraction")
            break
    else:
        if cost < stop_optimizing_if_mean_error_is_below:
            print("Stopping optimization because the cost (% of MVC error) has reached a satisfying level")
            break

plt.subplot(211)
plt.plot(best_output_force, color = 'red', linewidth = 3.5, label = 'best output force')
plt.plot(target_force, color = 'black', linestyle = ':', linewidth = 3, label = 'target force')
plt.xlabel("Time (ms)")
plt.ylabel("Torque (% MVC)")
plt.legend()
plt.title(f"Optimization of excitatory input signal (1st pass) \n Dark = first iterations; Light = last iterations")
plt.subplot(212)
plt.plot(optimized_excit_signal, color = 'red', linewidth = 3.5, label = "Best excitatory input")
plt.xlabel("Time (ms)")
plt.ylabel("Excitatory input (milliSiemens)")
plt.legend()
new_filename = f'Optimization_of_input_loss_1st_pass_force_output.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Plot error improvement
fig, ax = plt.subplots(figsize=(10,6))
plt.plot(errors)
plt.xlabel('Iteration')
plt.ylabel('Cost (mean error in % of MVC)')
plt.ylim([0,np.ceil(max(errors))])
plt.title('Cost Improvement Over Iterations (first pass)')
plt.grid(True)
new_filename = f'Optimization_of_input_loss_1st_pass.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [151]:
# Define the softmax function
def softmax_with_temperature(logits, temperature=1.0):
    """
    Compute the softmax of a list of logits with a temperature parameter.

    Parameters:
    logits (list or numpy array): The input logits.
    temperature (float): The temperature parameter.

    Returns:
    numpy array: The softmax probabilities.
    """
    # Convert logits to numpy array if they are not already
    logits = np.array(logits)
    
    # Apply the temperature parameter
    logits = logits / temperature
    
    # Compute the exponentials of the scaled logits
    exp_logits = np.exp(logits - np.max(logits))  # Subtract max for numerical stability
    
    # Compute the softmax probabilities
    softmax_probs = exp_logits / np.sum(exp_logits)
    
    return softmax_probs

In [ ]:
# SANITY CHECK ###############
plt.figure()
plt.plot(output_force) # sanity check before the rest of the optimization process
plt.ylabel("Force (% MVC)")

### Restrict window of analyzis
samples_for_analyzis = []
if target_type == 'trapezoid':
    if analyzis_window == 'plateau':
        samples_for_analyzis = np.arange((window_beginning_ignore+ramp_duration)*fsamp,len(time)-(window_end_ignore+ramp_duration)*fsamp)
    else:
        samples_for_analyzis = np.arange(window_beginning_ignore*fsamp,len(time)-window_end_ignore*fsamp)
else:
    samples_for_analyzis = np.arange(window_beginning_ignore*fsamp,len(time)-window_end_ignore*fsamp)

### Get discharge characteristics of motoneurons => only during window of analysis
# Retrieve spikes
spike_trains = []
for mni in range(nb_motoneurons_full_pool):
    spike_trains.append(monitor_spikes_motoneurons.spike_trains()[mni])
    # remove samples out of the analyzis window
    spike_trains[mni] = spike_trains[mni]/second # convert into a simple numpy array first
    spike_trains[mni] = spike_trains[mni][(spike_trains[mni] > (samples_for_analyzis[0]/fsamp)) & (spike_trains[mni] < (samples_for_analyzis[len(samples_for_analyzis)-1]/fsamp))]
    # spike_trains[mni] = spike_trains[mni]*second # convert back into a Brian2 array with unit
# Calculate the firing rate for each neuron
firing_rates = []
highest_ISIs = []
for mni in range(nb_motoneurons_full_pool):
    if len(spike_trains[mni]) <= 1:
        highest_ISIs.append(len(samples_for_analyzis)/fsamp)
    else:
        highest_ISIs.append(max(diff(spike_trains[mni])))
    # firing_rate_temp = len(spike_trains[mni]) / (len(samples_for_analyzis)/fsamp)
    if len(spike_trains[mni]) > 1:
        firing_rate_temp = 1/np.mean(np.diff(spike_trains[mni]))
    else:
        firing_rate_temp = 0
    firing_rates.append(firing_rate_temp)
# Convert to a numpy array for easier calculations
firing_rates = np.array(firing_rates)
### SELECT ONLY Selected MOTONEURONS
discontinuous_MUs_idx = {}
valid_MUs_idx = {}
# Index of discontinuous MNs (ISIs > threshold)
discontinuous_MUs_idx = [i for i, x in enumerate(highest_ISIs) if x > ISI_threshold_for_discontinuity]
discontinuous_MUs_idx = append(discontinuous_MUs_idx,
                            [i for i, x in enumerate(np.arange(nb_motoneurons_full_pool)) if len(spike_trains[x])<20]) # remove MUs with less than X spikes
discontinuous_MUs_idx = unique(discontinuous_MUs_idx)
valid_MUs_idx = [i for i, x in enumerate(arange(nb_motoneurons_full_pool)) if x not in discontinuous_MUs_idx]
print("Number of invalid MUs = ", len(discontinuous_MUs_idx), " out of ", nb_motoneurons_full_pool)
if motor_unit_subsampling_probability_distribution == 'uniform':
    sampling_probability_distribution = np.ones(shape(valid_MUs_idx))/len(valid_MUs_idx)
else:
    sampling_probability_distribution = np.copy(motoneuron_soma_diameters[valid_MUs_idx])
    sampling_probability_distribution = softmax_with_temperature(sampling_probability_distribution, bias_towards_larger_motor_neurons_temperature)

if subsample_MUs_for_analysis == False:
    selected_motor_units = valid_MUs_idx.copy()
    plot_title_text = '(all valid motor units selected)'
    txt_for_legend = f'selected motor units (n={len(valid_MUs_idx)}=all continuously active motor units)'
    plot_title_suffix = f' (all valid motor units)'
else:
    selected_motor_units = np.random.choice(valid_MUs_idx, size=nb_of_MUs_to_subsample, p=sampling_probability_distribution, replace=False)
    plot_title_text = f' (subset of {nb_of_MUs_to_subsample} motor units selected)'
    txt_for_legend = f'selected motor units (n={nb_of_MUs_to_subsample})'
    plot_title_suffix = f' (sampling of {nb_of_MUs_to_subsample} motor units)'
# selected_motor_units_relative_to_valid_MUs = np.array([np.where(valid_MUs_idx == x)[0][0] for x in selected_motor_units])
selected_motor_units_relative_to_valid_MUs = [valid_MUs_idx.index(x) for x in selected_motor_units]
# Firing rate results - only selected MNs
fig, axs = plt.subplots()
axs.hist(firing_rates[selected_motor_units], edgecolor='white', alpha=0.75)
mean_firing_rate_valid = np.mean(firing_rates[selected_motor_units])
std_firing_rate_valid = np.std(firing_rates[selected_motor_units])
axs.axvline(x = mean_firing_rate_valid, linestyle='--', linewidth=2, label='Mean firing rate')
axs.set_xlabel("Mean firing rate (pps)")
axs.set_ylabel("Motoneuron count count")
plt.tight_layout(rect=[0,0,1,0.96])
plt.suptitle("Histogram of motoneurons' firing rate" + plot_title_suffix)
new_filename = f'Hist_MN_Discharge_rates.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)
# Get recruitment threshold and discharge rates of motor units = only if chosing the "trapezoid" force target (because allows for gradual recruitment of motor units)
if target_type == 'trapezoid':
    MN_mean_firing_rates = np.copy(firing_rates) # all motoneurons by index, regardless of whether they are valid or not
    # Discharge rates duging the window of analyzis (the plateau section of the trapezoid)
    MN_recruitment_thresholds = np.full(firing_rates.shape, np.nan) # in % MVC
    spike_trains_full, binary_spike_trains_full = Get_binary_spike_trains(monitor_spikes_motoneurons, duration_with_ignored_window)
    for mni in range(nb_motoneurons_full_pool): #valid_MUs_idx:
        # RT as force during the median time of the first 3 firings
        temp_RT = (spike_trains_full[mni]/second)*fsamp
        temp_RT = np.median(temp_RT[0:2])
        if np.isnan(temp_RT)==False:
            MN_recruitment_thresholds[mni] = output_force[int(np.round(temp_RT))]
        # RT as force when the MN starts discharging with a rate > 5 pps (first ISI < 0.2)
        # MN_recruitment_thresholds[mni] = nan
        # spike_count = 0
        # for ISI_i in np.diff(spike_trains_full[mni]):
        #     if ISI_i < ISI_threshold_for_RT*second:
        #         MN_recruitment_thresholds[mni] = output_force[int(np.round((spike_trains_full[mni][spike_count]/second)*fsamp))]
        #         break
        #     spike_count += 1
MN_recruitment_thresholds_by_force_only_selected_idx = np.copy(MN_recruitment_thresholds)
MN_recruitment_thresholds_by_force_only_selected_idx = MN_recruitment_thresholds_by_force_only_selected_idx[selected_motor_units].reshape(-1, 1)
MN_mean_firing_rates_only_selected_idx = np.copy(MN_mean_firing_rates)
MN_mean_firing_rates_only_selected_idx = MN_mean_firing_rates[selected_motor_units].reshape(-1, 1)
# histogram of recruitment thresholds
plt.figure()
plt.hist(MN_recruitment_thresholds_by_force_only_selected_idx, density=True,
         edgecolor='white', alpha=0.75, color='C1')
plt.xlim(0,target_force_level*1)
ylabel("Proportion")
xlabel("Recruitment threshold (% MVC)")
title("Histogram of recruitment tresholds" + plot_title_suffix)
new_filename = f'RT_hsitogram.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [ ]:
# Create common noise (fluctuation of the excitatory input)
if excitatory_input_source  == 'generate_synthetic_input':
    common_noise = randn(len(time)) # noise input, with mean zero and std 1 (default setting)
    common_noise = lowpass_filter(common_noise, low_pass_filter_of_excitatory_input, fsamp)
    # remove artifacts of low pass filter
    common_noise[0:window_beginning_ignore*fsamp] = 0
    common_noise[len(common_noise)-(window_end_ignore*fsamp):len(common_noise)] = 0
elif excitatory_input_source == 'load_synthetic_input':
    synthetic_signals_dataframe = pd.read_csv(excitatory_input_sourcefile)
    common_noise = synthetic_signals_dataframe[f'{synthetic_signals_dataframe.shape[1]-1}'].values
    if excitatory_input_sourcefile_fsamp != fsamp:
        from scipy.interpolate import interp1d
        # Calculate the time array for the original signal
        loaded_signal_time = np.arange(len(synthetic_signals_dataframe)) / excitatory_input_sourcefile_fsamp
        # Calculate the number of samples in the resampled signal
        number_of_samples = int(len(synthetic_signals_dataframe) * fsamp / excitatory_input_sourcefile_fsamp)
        # Calculate the time array for the resampled signal
        resampled_time = np.linspace(loaded_signal_time[0], loaded_signal_time[-1], number_of_samples)
        # Create an interpolation function
        resample_loaded_signal_function = interp1d(loaded_signal_time, common_noise, kind='linear')
        common_noise = resample_loaded_signal_function(resampled_time)
    common_noise = common_noise[:len(time)] # cut the signal for it to be the right size
# Normalize
common_noise = ((common_noise - np.mean(common_noise)) / np.std(common_noise)) * excitatory_input_std
            
plt.figure()
plt.plot(common_noise, label="common noise / excitatory input fluctuations", color='C3')
plt.xlabel("Time (ms)")
plt.ylabel("Common noise amplitude (excitatory input fluctuations)")
plt.legend()
new_filename = f'Common_noise_signal.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

plt.figure()
N = len(common_noise)
yf = fft(common_noise)
xf = fftfreq(N, 1 / fsamp)
power_spectrum_temp = (np.abs(yf[:N//2])**2) / N
common_noise_power_integral = np.sum(power_spectrum_temp)
idx_corresponding_to_5hz = int(np.round((N/fsamp)*5))
power_spectrum_temp_0_5hz = power_spectrum_temp[:idx_corresponding_to_5hz]
common_noise_power_intergal_0_5_hz = np.sum(power_spectrum_temp_0_5hz)
plt.plot(xf[:N//2], power_spectrum_temp, color = 'C3', alpha = 1)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.title("Power spectrum of the common noise (fluctuations in common input)")
plt.xlim([0,10])
new_filename = f'Common_noise_power_spectrum.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [ ]:
# # Re-run the simulation with the parameter ending up with the lowest cost, with a low-pass filter of the optimized signal + common noise
# lowpass_filtered_optimized_excit_signal = lowpass_filter(optimized_excit_signal,1,fsamp) # 1hz low-pass filter
# # remove artifacts of low pass filter
# lowpass_filtered_optimized_excit_signal[0:int(np.round(window_beginning_ignore*fsamp)*0.5)] = 0
# lowpass_filtered_optimized_excit_signal[len(time)-int(np.round((window_end_ignore*fsamp)*0.5)):len(time)] = 0
# final_mean_excit_input = lowpass_filtered_optimized_excit_signal + common_noise

final_mean_excit_input = optimized_excit_signal + common_noise
input_inhib = TimedArray(inhib_input_per_MN * msiemens, dt=1*ms)

excit_input_per_MN = np.zeros((nb_motoneurons_full_pool,len(time)))
for mni in range(nb_motoneurons_full_pool):
    excit_input_per_MN[mni,:] = final_mean_excit_input * motoneurons_excitation_weights[mni] # scales both the mean excitation and common noise
    excit_input_per_MN[mni,:] += independent_noise_excit[mni,:]
    excit_input_per_MN[mni,:] = np.clip(excit_input_per_MN[mni,:], a_min=0, a_max=None)# Clamp to 0 to avoid negative conductance
excit_input_per_MN = np.transpose(excit_input_per_MN)
input_excit = TimedArray(excit_input_per_MN * msiemens, dt=1*ms)

eqs_motoneuron = LIF_equations

# Groups of neurons
motoneurons = NeuronGroup(nb_motoneurons_full_pool, eqs_motoneuron,
                        threshold='v>voltage_thresh',
                        reset='v=voltage_rest',
                        refractory='refractory_period',
                        method=sim_method)
                        
# Initialize values
motoneurons.v = voltage_rest # in mV #
motoneurons.g_leak = motoneurons_membrane_conductance * msiemens # in milisiemens
motoneurons.C_m = motoneuron_capacitances * ufarad # in microfarads
motoneurons.I_th = motoneurons_rheobases * rheobase_scaling * nA # in nanoAmperes
motoneurons.refractory_period = motoneurons_refractory_periods * ms  # in milliseconds
motoneurons.input_weight = motoneuron_input_weights # dimensionless unit
# Monitors
monitor_spikes_motoneurons = SpikeMonitor(motoneurons, record=True)

# Run simulation
run(duration_with_ignored_window)

# Get spike trains
spike_trains, binary_spike_trains = Get_binary_spike_trains(monitor_spikes_motoneurons, duration_with_ignored_window)

# Get force
output_force = Convolve_to_get_force(binary_spike_trains,np.arange(nb_motoneurons_full_pool),'normalized')
if low_pass_filter_force:
    output_force = lowpass_filter(output_force, low_pass_filter_of_force_cutoff, fsamp)

In [155]:
### Restrict window of analyzis
cut_edges_of_plateau = 1 # in s
samples_for_analyzis = []
if target_type == 'trapezoid':
    if analyzis_window == 'plateau':
        samples_for_analyzis = np.arange((window_beginning_ignore+ramp_duration+cut_edges_of_plateau)*fsamp,len(time)-(window_end_ignore+ramp_duration+cut_edges_of_plateau)*fsamp)
    else:
        samples_for_analyzis = np.arange(window_beginning_ignore*fsamp,len(time)-window_end_ignore*fsamp)
else:
    samples_for_analyzis = np.arange((window_beginning_ignore+cut_edges_of_plateau)*fsamp,len(time)-window_end_ignore*fsamp)

In [ ]:
# Plot target vs output force (initial and final)
fig, ax = plt.subplots(figsize=(20,7))
plt.plot(target_force, label='Target Force')
plt.plot(initial_output_force, label='Initial Output Force')
if keep_force_constant_despite_inhib:
    plt.plot(output_force, label='Final Output Force (optimization of common excitatory input taking inhibition into account)')
else:
    plt.plot(output_force, label='Final Output Force (optimization of common excitatory ignoring inhibition = so force should be lower)')
plt.xlabel('Time (ms)')
plt.ylabel('Force')
plt.legend()
plt.title('Force Output of Motor Neurons')
plt.grid(True)
new_filename = f'Optimized_input_resulting_force.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Plot control signal before and after learning
fig, ax = plt.subplots(figsize=(20,7))
plt.plot(initial_excit_signal, label='Initial Control Signal')
if keep_force_constant_despite_inhib:
    optimized_signal_label = 'Optimized Control Signal'
    final_mean_excit_input_signal_label = 'Common input (optimized control signal + common noise)'
else:
    optimized_signal_label = 'Optimized Control Signal (ignoring inhibition which is applied later)'
    final_mean_excit_input_signal_label = 'Common input (optimized control signal ignoring inhibition + common noise)'
plt.plot(optimized_excit_signal, label=optimized_signal_label)
plt.plot(np.clip(final_mean_excit_input, a_min=0, a_max=None), label=final_mean_excit_input_signal_label)
if nb_inhibitory_input >= 1:
    for inhibiti in range(nb_inhibitory_input):
        plt.plot(inhib_input[inhibiti][0], label=f'inhibitory input #{inhibiti+1}', color='C4', alpha = 1/nb_inhibitory_input)
plt.xlabel('Time (ms)')
plt.ylabel('Control Signal')
plt.legend()
# plt.ylim(0.5,2.5)
plt.title('Control Signal Before and After Learning + common input')
plt.grid(True)
new_filename = f'Optimized_input_&_final_mean_excit_input.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

print(f'Mean and std of common input during window of analyzis = {np.round(np.mean(final_mean_excit_input[samples_for_analyzis])*100)/100} +/- {np.round(np.std(final_mean_excit_input[samples_for_analyzis])*100)/100}')

# Getting a smooth color blend from a given colormap
colormap_temp = cm.get_cmap('plasma')

# Raster plot of discharge times
plt.figure(num=1,figsize=(20,10))
for mni in range(nb_motoneurons_full_pool):
    plt.scatter((spike_trains[mni]/second)*fsamp, np.ones(len(spike_trains[mni]))*mni, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), linewidth = 0.2, alpha = 0.3)
plt.vlines(samples_for_analyzis[0],plt.ylim()[0],plt.ylim()[1],color='black',label='Start of the analyzis window')
plt.vlines(samples_for_analyzis[len(samples_for_analyzis)-1],plt.ylim()[0],plt.ylim()[1],color='black',label='End of the analyzis window')
plt.xlabel('Time (s)')
plt.ylabel('Motoneuron index')
plt.title("Raster plot of motoneuron spikes \n Opaque = continuous MU; transparent = discontinuous MU")
plt.legend()
new_filename = f'Optimized_input_firings_raster_plot.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Force per MU
force_per_MU = []
force_total = zeros(len(binary_spike_trains[0,:]))
fig, ax = plt.subplots(figsize=(25, 10))

for mni in range(nb_motoneurons_full_pool):
    # 'same' mode means the output length will be the same as the input length
    if len(spike_trains[mni]) >= 1: # at least one spike necessary
        temp_force = Convolve_to_get_force(np.reshape(binary_spike_trains[mni,:], (1, len(binary_spike_trains[mni,:]))), [mni], 'normalized')
        force_per_MU.append(temp_force)
        ax.plot(force_total, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5, linewidth = 0.5)
        force_total = force_total + temp_force
if low_pass_filter_force:
    ax.plot(output_force, color='black', alpha = 1, linewidth = 2, label = "Total force (low-pass filtered)")
else:
    ax.plot(force_total, color='black', alpha = 1, linewidth = 2, label = "Total force")
plt.title(f"Reconstructed force (convolving spike train with twitch force kernel)")
plt.ylabel("Force (% MVC)")
plt.xlabel("Time (ms)")
plt.legend()
new_filename = f'Optimized_input_cumulative_force_per_MU.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)
 

In [ ]:
### Get discharge characteristics of motoneurons => only during window of analysis
fig, axs = plt.subplots()
# Retrieve spikes
spike_trains = []
for mni in range(nb_motoneurons_full_pool):
    spike_trains.append(monitor_spikes_motoneurons.spike_trains()[mni])
    # remove samples out of the analyzis window
    spike_trains[mni] = spike_trains[mni]/second # convert into a simple numpy array first
    spike_trains[mni] = spike_trains[mni][(spike_trains[mni] > (samples_for_analyzis[0]/fsamp)) & (spike_trains[mni] < (samples_for_analyzis[len(samples_for_analyzis)-1]/fsamp))]
    # spike_trains[mni] = spike_trains[mni]*second # convert back into a Brian2 array with unit
# Calculate the firing rate for each neuron
firing_rates = []
highest_ISIs = []
for mni in range(nb_motoneurons_full_pool):
    if len(spike_trains[mni]) <= 1:
        highest_ISIs.append(len(samples_for_analyzis)/fsamp)
    else:
        highest_ISIs.append(max(diff(spike_trains[mni])))
    # firing_rate_temp = len(spike_trains[mni]) / (len(samples_for_analyzis)/fsamp)
    if len(spike_trains[mni]) > 1:
        firing_rate_temp = 1/np.mean(np.diff(spike_trains[mni]))
    else:
        firing_rate_temp = 0
    firing_rates.append(firing_rate_temp)
# Convert to a numpy array for easier calculations
firing_rates = np.array(firing_rates)
# Calculate mean and standard deviation of the firing rates
mean_firing_rate = np.mean(firing_rates)
std_firing_rate = np.std(firing_rates)
# Motoneurons' firing rates results
axs.hist(firing_rates, edgecolor='white', alpha=0.75)
axs.axvline(x = mean_firing_rate, linestyle='--', linewidth=2, label='Mean firing rate')
axs.set_xlabel("Mean firing rate (pps)")
axs.set_ylabel("Motoneuron count count")
plt.tight_layout(rect=[0,0,1,0.96])
plt.suptitle("Histogram of motoneurons' firing rate (all motoneurons, only during window of analyzis)")

### SELECT ONLY Selected MOTONEURONS
discontinuous_MUs_idx = {}
valid_MUs_idx = {}
fig, axs = plt.subplots()

# Index of discontinuous MNs (ISIs > threshold)
discontinuous_MUs_idx = [i for i, x in enumerate(highest_ISIs) if x > ISI_threshold_for_discontinuity]
discontinuous_MUs_idx = append(discontinuous_MUs_idx,
                            [i for i, x in enumerate(np.arange(nb_motoneurons_full_pool)) if len(spike_trains[x])<20]) # remove MUs with less than X spikes
discontinuous_MUs_idx = unique(discontinuous_MUs_idx)

valid_MUs_idx = [i for i, x in enumerate(arange(nb_motoneurons_full_pool)) if x not in discontinuous_MUs_idx]
print("Number of invalid MUs = ", len(discontinuous_MUs_idx), " out of ", nb_motoneurons_full_pool)
axs.hist(highest_ISIs, edgecolor='white', alpha=0.5)
axs.axvline(x = np.median(highest_ISIs), linestyle='--', linewidth=3, label='Mean highest ISI')
axs.axvline(x = ISI_threshold_for_discontinuity, color = 'black', linestyle='-', linewidth=2, alpha = 0.5, label='Mean firing rate')
axs.set_xlabel("Max ISI (s)")
axs.set_ylabel("Motoneuron count")
plt.tight_layout(rect=[0,0,1,0.92])
plt.suptitle("Histogram of motoneurons' max ISI \n (colored line is median ; black line is threshold)")
# new_filename = f'Hist_MN_ISIs.png'
# save_file_path = os.path.join(new_directory, new_filename)
# plt.savefig(save_file_path)
plt.show(fig)

In [ ]:
# SELECT A SUBSET OF MOTOR UNITS
plot_title_text = 'sampling of motor units for analysis' 

if motor_unit_subsampling_probability_distribution == 'uniform':
    sampling_probability_distribution = np.ones(shape(valid_MUs_idx))/len(valid_MUs_idx)
else:
    sampling_probability_distribution = np.copy(motoneuron_soma_diameters[valid_MUs_idx])
    sampling_probability_distribution = softmax_with_temperature(sampling_probability_distribution, bias_towards_larger_motor_neurons_temperature)

if subsample_MUs_for_analysis == False:
    selected_motor_units = valid_MUs_idx.copy()
    plot_title_text = plot_title_text + ' (all valid motor units selected)'
    txt_for_legend = f'selected motor units (n={len(valid_MUs_idx)}=all continuously active motor units)'
    plot_title_suffix = f' (all valid motor units)'
else:
    selected_motor_units = np.random.choice(valid_MUs_idx, size=nb_of_MUs_to_subsample, p=sampling_probability_distribution, replace=False)
    plot_title_text = plot_title_text + f' (subset of {nb_of_MUs_to_subsample} motor units selected)'
    txt_for_legend = f'selected motor units (n={nb_of_MUs_to_subsample})'
    plot_title_suffix = f' (sampling of {nb_of_MUs_to_subsample} motor units)'
# selected_motor_units_relative_to_valid_MUs = np.array([np.where(valid_MUs_idx == x)[0][0] for x in selected_motor_units])
selected_motor_units_relative_to_valid_MUs = [valid_MUs_idx.index(x) for x in selected_motor_units]

plt.figure(figsize=(20,5))
plt.bar(valid_MUs_idx,sampling_probability_distribution, label='probability of each valid (continuously firing) MU', color='C9', alpha=0.3)
plt.bar(selected_motor_units,sampling_probability_distribution[selected_motor_units_relative_to_valid_MUs], label=txt_for_legend, color='C9')
plt.legend()
plt.ylabel("Probability")
plt.xlabel("Motor unit index")
plt.title(plot_title_text)

new_filename = f'Valid_VS_sampled_motor_units_for_analyzis.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

In [ ]:
# Firing rate results - only selected MNs
fig, axs = plt.subplots()
axs.hist(firing_rates[selected_motor_units], edgecolor='white', alpha=0.75)
mean_firing_rate_valid = np.mean(firing_rates[selected_motor_units])
std_firing_rate_valid = np.std(firing_rates[selected_motor_units])
axs.axvline(x = mean_firing_rate_valid, linestyle='--', linewidth=2, label='Mean firing rate')
axs.set_xlabel("Mean firing rate (pps)")
axs.set_ylabel("Motoneuron count count")
plt.tight_layout(rect=[0,0,1,0.96])
plt.suptitle("Histogram of motoneurons' firing rate" + plot_title_suffix)
new_filename = f'Discharge_rates_Hist.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

plt.figure(figsize=(10,10))
plt.subplot(211)
plt.plot(np.arange(nb_motoneurons_full_pool)[selected_motor_units], firing_rates[selected_motor_units], color = 'C0', linewidth = 3)
plt.ylabel("Firing rate (pps)")
plt.xlabel("MN index")
plt.title("Firing rates (only selected MUs) ~ MN index")
plt.subplot(212)
plt.plot(motoneuron_soma_diameters[selected_motor_units], firing_rates[selected_motor_units], color = 'C0', linewidth = 3)
plt.ylabel("Firing rate (pps)")
plt.xlabel("soma size (micrometers)")
plt.title("Firing rates (only selected MUs) ~ soma diameter")
new_filename = f'Discharge_rates.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)
    

In [ ]:
## SMOOTHING SPIKE TRAINS

Wind_s = 0.4  # hanning window duration. 0.4 for 2.5hz low-pass, 0.2 for 5hz low-pass
HanningW = 2 / round(fsamp * Wind_s) * windows.hann(round(fsamp * Wind_s))  # unitary area

# Filter all valid motor units
smoothed_signal = []
for mni in range(nb_motoneurons_full_pool):
    if mni in valid_MUs_idx:
        smoothed_signal.append(filtfilt(HanningW, 1, binary_spike_trains[mni, :] * fsamp))
smoothed_signal = np.array(smoothed_signal)

fig, ax = plt.subplots(figsize=(25, 10))
colormap_temp = cm.get_cmap('plasma') # Getting a smooth color blend from a given colormap
if subsample_MUs_for_analysis == True:
    for mni in range(len(valid_MUs_idx)):
        ax.plot((smoothed_signal)[mni,:], color=colormap_temp(valid_MUs_idx[mni]/(nb_motoneurons_full_pool-1)), alpha = 0.3)
# Remove non-valid motor units, and replot on top of the previous plot only the sampled motor units
smoothed_signal = smoothed_signal[selected_motor_units_relative_to_valid_MUs,:]
for mni in range(smoothed_signal.shape[0]):
    ax.plot((smoothed_signal)[mni,:], color=colormap_temp(selected_motor_units[mni]/(nb_motoneurons_full_pool-1)), alpha = 1)

plt.title(f"Smoothed signals of only continuous MUs \n (dark = small MNs ; light = large MNs) \n (low opacity = valid but not selected ; opaque = selected motor units)")
plt.ylabel("Smoothed discharge rate (pps)")
plt.xlabel("Time (ms)")
plt.vlines(samples_for_analyzis[0],plt.ylim()[0],plt.ylim()[1],color='black',label='Start of the analyzis window')
plt.vlines(samples_for_analyzis[len(samples_for_analyzis)-1],plt.ylim()[0],plt.ylim()[1],color='black',label='End of the analyzis window')
plt.legend()
new_filename = f'Smoothed_discharge_rates_only_continuous_MUs.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

# restrict signal to window of analyzis, with only valid motor units
smoothed_signal = smoothed_signal[:,samples_for_analyzis]
# re-plot
fig, ax = plt.subplots(figsize=(25, 10))
for mni in range(smoothed_signal.shape[0]):
    ax.plot(smoothed_signal[mni,:], color=colormap_temp(selected_motor_units[mni]/(nb_motoneurons_full_pool-1)))
plt.title(f"Smoothed signals only during window of analyzis" + plot_title_suffix + "\n (dark = small MNs ; light = large MNs)")
plt.ylabel("Smoothed discharge rate (pps)")
plt.xlabel("Time (ms)")
new_filename = f'Smoothed_discharge_rates_only_continuous_MUs_window_of_analyzis.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

    

In [161]:
# # For debug/testing purpose

# smoothed_signal_normalized = np.copy(smoothed_signal)
# for mni in range(smoothed_signal.shape[0]):
#     print(f"Mean = {np.round(np.mean(smoothed_signal_normalized[mni])*100)/100}, STD = {np.round(np.std(smoothed_signal_normalized[mni])*100)/100}")

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

# PCA
smoothed_signal_normalized = np.copy(smoothed_signal)
for mni in range(smoothed_signal.shape[0]):
    smoothed_signal_normalized[mni] = smoothed_signal_normalized[mni]-np.mean(smoothed_signal_normalized[mni])
    # In some cases (when no noise is injected), the std can be very low and cause division by zero
    if np.std(smoothed_signal_normalized[mni]) > 0.01:
        smoothed_signal_normalized[mni] = smoothed_signal_normalized[mni]/np.std(smoothed_signal_normalized[mni])
    else:
        smoothed_signal_normalized[mni] = smoothed_signal_normalized[mni]/0.01

# Displaye smoothed signals normalized (mean of 0, std of 1)
fig, ax = plt.subplots(figsize=(25, 10))
for mni in range(smoothed_signal.shape[0]):
    ax.plot(smoothed_signal_normalized[mni,:], color=colormap_temp(selected_motor_units[mni]/(nb_motoneurons_full_pool-1)))
plt.title(f"Normalized smoothed signals of only continuous MUs, only during window of analyzis"  + plot_title_suffix + "\n (dark = small MNs ; light = large MNs)")
plt.ylabel("Normalizd smoothed discharge rate (in std)")
plt.xlabel("Time (ms)")
new_filename = f'Smoothed_discharge_rates_ormalized_only_continuous_MUs_window_of_analyzis.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

nb_PCs_to_store = 5

# Function to calculate R-squared for each time series with a given number of PCs
def calculate_r_squared(data, pca, num_pcs):
    # Transform the data using the selected number of PCs
    transformed_data = pca.transform(data)
    # Inverse transform to reconstruct the data
    reconstructed_data = pca.inverse_transform(
        np.hstack([transformed_data, np.zeros((data.shape[0], pca.n_components_ - num_pcs))])
    )
    # Calculate R-squared for each time series (motor unit)
    r_squared_values = [r2_score(data[:, i], reconstructed_data[:, i]) for i in range(data.shape[1])]
    return r_squared_values

# Assume `smoothed_discharge_rates` is the variable holding     the data
# Perform PCA
nb_PCs = 10 # nb_motoneurons_full_pool
pca = PCA(n_components=nb_PCs)
pca_result = pca.fit_transform(transpose(smoothed_signal_normalized))

# Initialize a DataFrame to store the R-squared values
PCA_r_squared_df = pd.DataFrame(index=range(1, nb_PCs_to_store + 1), columns=range(smoothed_signal_normalized.shape[0])) # nb_PCs + 1
# Calculate R-squared values for 1 to 2 PCs
for num_pcs in range(1,nb_PCs_to_store+1): # From 1 to 2 PCs # nb_PCs + 1): 
    pca_temp = PCA(n_components=num_pcs)
    pca_temp.fit(smoothed_signal_normalized.T)
    r_squared_values = calculate_r_squared(smoothed_signal_normalized.T, pca_temp, num_pcs)
    PCA_r_squared_df.loc[num_pcs] = r_squared_values

# Display the explained variance ratio (proportion of variance explained by each PC)
explained_variance_ratio = pca.explained_variance_ratio_
# Optionally, display the cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Plot the explained variance
plt.figure(figsize=(8, 6))
plt.bar(range(0, nb_PCs + 1), append(0,explained_variance_ratio), alpha=0.5, align='center',
        label='Individual explained variance')
plt.plot(range(0, nb_PCs + 1), append(0,cumulative_explained_variance),
         label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.title('Explained Variance by Principal Components' + plot_title_suffix)
plt.ylim(-0.1,1.1)
plt.legend(loc='best')
plt.grid(True)
new_filename = f'Error_correction_PCA_VAF.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

print(f'Mean R² of PC1+PC2 = {np.mean(PCA_r_squared_df.iloc[1])}')
print(f'Cumulative VAF for PC1+PC2 = {cumulative_explained_variance[1]}')

In [ ]:
from scipy.signal import csd, detrend

windowCOH = 1 # in seconds
frequencies_per_FFT_window = 10
indexes_of_windows_below_5hz = np.arange(0,5*frequencies_per_FFT_window)

# From groups of 2 up to half of the motor neurons
shuffled_idx_list = list.copy(list(selected_motor_units)) # initialize list of indices to be shuffled iteratively
window_analyzis_begin = int(samples_for_analyzis[0])
window_analyzis_end = int(samples_for_analyzis[len(samples_for_analyzis)-1])
seg_pwr = 10 # segment length (for coherence analyzis), specificied as a power of 2. 2^10 is ~1s for a fsamp of 1000

max_or_mean_0_5hz_COH = 'max'

COH_calc_group_size_nb = int(floor(len(selected_motor_units)/2)-1)
# COH_calc_max_iteration_nb_per_group_size = 1000 # More iteration for smaller group sizes, because the value obtained is very dependent upon the exact neurons selected, especially when only a few MNs are used to create the CST
COH_0_5hz_per_group = []
COH_mean_0_5hz = np.zeros(COH_calc_group_size_nb)
COH_pooled_per_group_size = []

colors_plots = plt.cm.winter(np.linspace(0, 1, COH_calc_group_size_nb))
plt.figure(figsize=(10,8))
for group_sizi in range(COH_calc_group_size_nb):
    coherence_temp = []
    COH_calc_iteration_nb_per_group_size_temp = int(np.max([np.round(COH_calc_max_iteration_nb_per_group_size/(group_sizi+1)),1]))
    COH_0_5hz_per_group.append([])
    print(f'Iterating for groups of {group_sizi+1} motoneurons (out of a max group size of {COH_calc_group_size_nb}) - {COH_calc_iteration_nb_per_group_size_temp} iterations')
    for group_iteri in range(COH_calc_iteration_nb_per_group_size_temp):
        random.shuffle(shuffled_idx_list)
        idx_cst1 = np.copy(shuffled_idx_list[0:group_sizi+1])
        cst1 = sum(binary_spike_trains[idx_cst1,window_analyzis_begin:window_analyzis_end],axis=0)
        idx_cst2 = np.copy(shuffled_idx_list[len(shuffled_idx_list)-(group_sizi+1):len(shuffled_idx_list)])
        cst2 = sum(binary_spike_trains[idx_cst2,window_analyzis_begin:window_analyzis_end],axis=0)

        # Compute intra-group coherence for group 1
        f, COH_intragroup_X = csd(detrend(cst1), detrend(cst1), window=windows.hann(round(windowCOH * fsamp)), noverlap=0, nfft=frequencies_per_FFT_window * fsamp, fs=fsamp)
        # Compute intra-group coherence for group 2
        f, COH_intragroup_Y = csd(detrend(cst2), detrend(cst2), window=windows.hann(round(windowCOH * fsamp)), noverlap=0, nfft=frequencies_per_FFT_window * fsamp, fs=fsamp)
        # Compute inter-group coherence
        f, COH_intergroup = csd(detrend(cst1), detrend(cst2), window=windows.hann(round(windowCOH * fsamp)), noverlap=0, nfft=frequencies_per_FFT_window * fsamp, fs=fsamp)

        coherence_temp.append( (np.abs(COH_intergroup) ** 2) / (COH_intragroup_X * COH_intragroup_Y) ) # Welch's method of coherence calculation
        if max_or_mean_0_5hz_COH == 'mean':
            # COH_0_5hz_per_group[group_sizi,group_iteri] = np.nanmean(coherence_temp[group_iteri][indexes_of_windows_below_5hz])
            COH_0_5hz_per_group[group_sizi].append(np.nanmean(coherence_temp[group_iteri][indexes_of_windows_below_5hz]))
        elif max_or_mean_0_5hz_COH == 'max':
            # COH_0_5hz_per_group[group_sizi,group_iteri] = np.nanmax(coherence_temp[group_iteri][indexes_of_windows_below_5hz])
            COH_0_5hz_per_group[group_sizi].append(np.nanmax(coherence_temp[group_iteri][indexes_of_windows_below_5hz]))

        # plt.scatter(group_sizi,COH_0_5hz_per_group[group_sizi,group_iteri],s=30,color=colors_plots[group_sizi],alpha=min(3/COH_calc_iteration_nb_per_group_size,1))
        plt.scatter(group_sizi,COH_0_5hz_per_group[group_sizi][group_iteri],s=30,color=colors_plots[group_sizi],alpha=min(np.sqrt(1/COH_calc_iteration_nb_per_group_size_temp),1))
    COH_pooled_per_group_size.append( np.nanmean(coherence_temp, axis=0) )
    # COH_mean_0_5hz[group_sizi] = np.nanmean(COH_0_5hz_per_group[group_sizi,:],axis=0)
    COH_mean_0_5hz[group_sizi] = np.nanmean(COH_0_5hz_per_group[group_sizi],axis=0)

plt.plot(COH_mean_0_5hz, linewidth=3, color='red', alpha=0.5, label=f"Mean of [max 0-5hz coherence for CSTs of X spike trains]")
plt.xlabel('Number of MNs in the CSTs')
plt.ylabel('Mean coherence in the 0-5hz bandwidth')
plt.title("Increase in coherence in the 0-5hz bandwidth (Y) as the number of CSTS' MNs (X) increase" + plot_title_suffix)
plt.ylim(0,1)
plt.legend()
new_filename = f'PCI_curve_of_0-5hz_coh.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

plt.figure(figsize=(10, 8))
for group_sizi in range(COH_calc_group_size_nb):
    plt.plot(COH_pooled_per_group_size[group_sizi], color=colors_plots[group_sizi], alpha = 0.5, linewidth = 2)
plt.xlim(0,20*frequencies_per_FFT_window)
plt.xticks(ticks=plt.xticks()[0], labels=[str(int(x / frequencies_per_FFT_window)) for x in plt.xticks()[0]])
plt.xlabel("Frequency (Hz)")
plt.ylabel("Coherence")
plt.ylim(0,1)
plt.title("Mean coherence" + plot_title_suffix)
new_filename = f'Coherence_curve.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

In [164]:
# root mean square error function definition
def rmse(predicted, true):
    # Root mean square
    return np.sqrt(np.mean((true - predicted) ** 2))
    # Mean square
    # return np.mean((true - predicted) ** 2)

In [ ]:
from scipy.optimize import curve_fit, least_squares

# Define the model - implementation from Negro et al 2016 https://physoc.onlinelibrary.wiley.com/doi/epdf/10.1113/JP271748 
def PCI_model(n, A, B):
    return abs(n**2 * A)**2 / ((n*B) + ((n**2)*A))**2
# Define the residuals function
def residuals(params, x, y):
    A, B = params
    return y - PCI_model(x, A, B)

plt.figure(figsize=(12,8))
n = np.arange(1,COH_mean_0_5hz.shape[0]+1)
# Negro 2016 equation:
# Mean COH in a given frequency band: Eqn 4 = abs(n**2 * A)**2 / ((n*B) + ((n**2)*A))**2
#   - n = number of neurons in the CST
#   - A = power of the common synaptic input (in the given frequency band), multipled by the absolute square of the susceptibility
#   - A = abs(X(f))**2 * Ss(f)
#       => X(f) is the response function (susceptibility) of the motor neuron [POOl of motor neurons in our case] to the stimulus and Ss(f) the power spectrum of the stimulus
#   - B = (power of the?) response of the pool with independent synaptic input
#   - B = Sn(f)
#       => Sn(f)is the power spectrum of the output spike train [CST of spike train in our case] when it is driven by independent synaptic input only
#   - proportion of common input (PCI) = estimate of gamma (common voltage fluctuation of membrane) = sqrt(A/(B+A)) = proportion of the common synaptic input with respect to the total synaptic input received by the motor neurons [total input can be inferred from motor pool output, because it the output is a function of both the common and independent input]
#       => They say just sqrt(A/B) in the paper, but this can result in a proportion (PCI) > 1 which shouldn't be possible, and I get results fitting very closely the theoritical values when computing sqrt(A_fit / (B_fit + A_fit))
#   - The ratio can be estimated by an experimental measure of the mean coherence in the given frequency range for varying n (number of motor neuron spike trains used in the calculation (Negro & Farina, 2012)) using eqn (4)
#   - Using a least-square curve fitting of the estimated values of coherence for CSTs with different numbers of motor neurons, the parameters A and B of eqn (4) can be estimated.

# params, covariance = curve_fit(PCI_model, xdata = n, ydata = COH_mean_0_5hz) # older version, still works well
initial_guess = [1, 1] # Initial guess for the parameters from which to optimize using least squares
least_square_optim_results = least_squares(residuals, initial_guess, loss='soft_l1', f_scale=0.1, args=(n, COH_mean_0_5hz)) # default loss function is 'linear' but 'soft_l1' is a bit more robust to fluctuations
# Extract parameters
A_fit, B_fit = least_square_optim_results.x
PCI_estimated = np.sqrt(A_fit/(B_fit+A_fit))
fitted_PCI_curve = PCI_model(n, A_fit, B_fit)

# Get ratio of common excitatory input to independent input = Grond-truth PCI
# = power spectrum integral of common input in the 0-5hz range over power spectrum integral of independent input in the same range
# Ignoring inhibitory input = hard to calculate, so no ground truth when simulating inhibition
if (nb_inhibitory_input >= 1) and (inhibitory_input_mean >= 1*1e-2) and (np.sum(MN_inhibition_weights[0]) >= 0.1): # if there is inhibition
    PCI_ground_truth_without_inhib = np.nan # not defined if there is inhibition
else: # else if no inhibition
    PCI_ground_truth_without_inhib = np.sqrt(common_noise_power_intergal_0_5_hz / (common_noise_power_intergal_0_5_hz + independent_excit_noise_power_integral_0_5_hz_mean))

plt.plot(COH_mean_0_5hz, color='C0',linewidth=2.5, alpha = 0.5, label='ground truth curve')
plt.plot(fitted_PCI_curve, color='blue',linewidth=2, alpha = 0.5, linestyle='dashed',label='fitted curve')
plt.xlabel('Number of MNs in the CSTs')
plt.ylabel('Mean coherence in the 0-5hz bandwidth')
plt.title(f"Increase in coherence in the 0-5hz bandwidth (Y) as the number of CSTS' MNs (X) increase => Ground-truth VS fitted data" + plot_title_suffix + f"\n Estimated PCI (gamma) = {np.round(PCI_estimated*1000)/1000} \n Ground-truth PCI (ratio of common input 0-5hz integral of power spectrum VS idem common+independent input) = {np.round(PCI_ground_truth_without_inhib*1000)/10}%, ignoring inhibition \n")
plt.ylim(0,1)
plt.legend()
new_filename = f'PCI_fitted_vs_true_curve.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [ ]:
# SIMON'S INSPIRED VERSION - Implementation from Negro et al 2016 https://physoc.onlinelibrary.wiley.com/doi/epdf/10.1113/JP271748 
plt.figure(figsize=(12,8))
n = np.arange(1,COH_mean_0_5hz.shape[0]+1)
# Negro 2016 equation:
# Mean COH in a given frequency band: Eqn 4 = (abs(n**2 * A)**2) / ((n*B) + (((n**2)*A))**2)
#   - n = number of neurons in the CST
#   - A = power of the common synaptic input (in the given frequency band), multipled by the absolute square of the susceptibility (which is 1 when considering the 0-5hz bandwidth I think)
#   - B = response of the pool with independent synaptic input (of the same bandwidth). Assumed to be 1 for the 0-5hz bandwidth.
#       (k in the fitting below correspond to sqrt(B), since)
#   #   #
#   - proportion of common input (PCI) = estimate of gamma (common voltage fluctuation of membrane) = sqrt(A/B) = proportion of the common synaptic input with respect to the total synaptic input received by the motor neurons
#   - The ratio can be estimated by an experimental measure of the mean coherence in the given frequency range for varying n (number of motor neuron spike trains used in the calculation (Negro & Farina, 2012)) using eqn (4)
#   - Using a least-square curve fitting of the estimated values of coherence for CSTs with different numbers of motor neurons, the parameters A and B of eqn (4) can be estimated.
nb_gammas_to_try = int(1e3) # Correspond to B
gamma_tmp = np.linspace(0.1,1e2,nb_gammas_to_try)
gamma_fit_error = []
for k in gamma_tmp:
    c = n**4 / (n / k**2 + n**2)**2
    plt.plot(c, color='C0',linewidth=1, alpha = min(5/nb_gammas_to_try,1))
    gamma_fit_error.append(rmse(c,COH_mean_0_5hz))
idx_with_min_error = gamma_fit_error.index(min(gamma_fit_error))
gamma = np.sqrt(gamma_tmp[idx_with_min_error]**2 / (gamma_tmp[idx_with_min_error]**2 +1))
fitted_PCI_curve = n**4 / ( n / gamma_tmp[idx_with_min_error]**2 + n**2 )**2
# fitted_PCI_curve = n**4 / ( n / gamma**2 + n**2 )**2

plt.plot(COH_mean_0_5hz, color='C0',linewidth=2.5, alpha = 0.5, label='ground truth curve')
plt.plot(fitted_PCI_curve, color='red',linewidth=2, alpha = 0.5, linestyle='dashed',label='fitted curve')
plt.xlabel('Number of MNs in the CSTs')
plt.ylabel('Mean coherence in the 0-5hz bandwidth')
plt.title(f"Increase in coherence in the 0-5hz bandwidth (Y) as the number of CSTS' MNs (X) increase => Ground-truth VS fitted data \n Estimated PCI (gamma) = {np.round(gamma*1000)/1000} \n Ground-truth PCI (ratio of common input variance VS independent input variance) = {np.round(PCI_ground_truth_without_inhib*1000)/10}%, ignoring inhibition \n")
plt.ylim(0,1)
plt.legend()
new_filename = f'PCI_fitted_vs_true_curve_SimonMethod.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [ ]:
# Get recruitment threshold and discharge rates of motor units = only if chosing the "trapezoid" force target (because allows for gradual recruitment of motor units)
if target_type == 'trapezoid':
    MN_mean_firing_rates = np.copy(firing_rates) # all motoneurons by index, regardless of whether they are valid or not
    MN_std_firing_rates = np.zeros(nb_motoneurons_full_pool)
    # Discharge rates duging the window of analyzis (the plateau section of the trapezoid)
    MN_recruitment_thresholds_by_force = np.full(firing_rates.shape, np.nan) # in % MVC
    MN_recruitment_thresholds_by_excitatory_input = np.full(firing_rates.shape, np.nan) # in milliSiemens of excitatory input signal
    spike_trains_full, binary_spike_trains_full = Get_binary_spike_trains(monitor_spikes_motoneurons, duration_with_ignored_window)
    for mni in range(nb_motoneurons_full_pool):
        # RT as force during the median time of the first 3 firings
        temp_RT = (spike_trains_full[mni]/second)*fsamp
        temp_RT = np.median(temp_RT[0:2])
        if np.isnan(temp_RT)==False:
            MN_recruitment_thresholds_by_force[mni] = output_force[int(np.round(temp_RT))]
            MN_recruitment_thresholds_by_excitatory_input[mni] = final_mean_excit_input[int(np.round(temp_RT))]
        # RT as force when the MN starts discharging with a rate > 5 pps (first ISI < 0.2)
        # MN_recruitment_thresholds[mni] = nan
        # spike_count = 0
        # for ISI_i in np.diff(spike_trains_full[mni]):
        #     if ISI_i < ISI_threshold_for_RT*second:
        #         MN_recruitment_thresholds[mni] = output_force[int(np.round((spike_trains_full[mni][spike_count]/second)*fsamp))]
        #         break
        #     spike_count += 1
        if len(spike_trains_full[mni]) > 1:
            MN_std_firing_rates[mni] = np.std(1/np.diff(spike_trains_full[mni]))
        else:
            MN_std_firing_rates[mni] = nan
                
MN_recruitment_thresholds_by_force_only_selected_idx = np.copy(MN_recruitment_thresholds_by_force)
MN_recruitment_thresholds_by_force_only_selected_idx = MN_recruitment_thresholds_by_force_only_selected_idx[selected_motor_units].reshape(-1, 1)
# MN_recruitment_thresholds_by_force_only_selected_idx = MN_recruitment_thresholds_by_force_only_selected_idx.reshape(-1, 1)
MN_recruitment_thresholds_by_input_only_selected_idx = np.copy(MN_recruitment_thresholds_by_excitatory_input)
MN_recruitment_thresholds_by_input_only_selected_idx = MN_recruitment_thresholds_by_input_only_selected_idx[selected_motor_units].reshape(-1, 1)
# MN_recruitment_thresholds_by_force_only_selected_idx = MN_recruitment_thresholds_by_force_only_selected_idx.reshape(-1, 1)
MN_mean_firing_rates_only_selected_idx = np.copy(MN_mean_firing_rates)
MN_mean_firing_rates_only_selected_idx = MN_mean_firing_rates[selected_motor_units].reshape(-1, 1)
# MN_mean_firing_rates_only_selected_idx = MN_mean_firing_rates.reshape(-1, 1)

# histogram of recruitment thresholds
plt.figure()
plt.hist(MN_recruitment_thresholds_by_force_only_selected_idx, density=True,
         edgecolor='white', alpha=0.75, color='C1')
plt.xlim(0,target_force_level*1)
ylabel("Proportion")
xlabel("Recruitment threshold (% MVC)")
title("Histogram of recruitment tresholds" + plot_title_suffix)
new_filename = f'RT_force_histogram.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# histogram of recruitment thresholds
plt.figure()
plt.hist(MN_recruitment_thresholds_by_input_only_selected_idx, density=True,
         edgecolor='white', alpha=0.75, color=[1,0.7,0])
ylabel("Proportion")
xlabel("Excitatory input (milliSiemens)")
title("Histogram of recruitment tresholds" + plot_title_suffix)
new_filename = f'RT_ecit_input_histogram.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

true_false_continuously_firing_MUs = np.zeros(nb_motoneurons_full_pool)
true_false_continuously_firing_MUs[valid_MUs_idx] = 1
true_false_sampled_motor_unit = np.zeros(nb_motoneurons_full_pool)
true_false_sampled_motor_unit[selected_motor_units] = 1

sampling_probability_dsitribution_to_save = np.zeros(nb_motoneurons_full_pool)
if subsample_MUs_for_analysis == True:
    sampling_probability_dsitribution_to_save[valid_MUs_idx] = sampling_probability_distribution
else:
    sampling_probability_dsitribution_to_save[valid_MUs_idx] = 1

# Save recruitment thresholds and firing rates
RT_mean_DR_dataframe = pd.DataFrame({
    'MU_idx': np.arange(nb_motoneurons_full_pool),
    'Continusouly_firing': true_false_continuously_firing_MUs,
    'Sampled_for_analyzis': true_false_sampled_motor_unit,
    'Recruitment_threshold_force': MN_recruitment_thresholds_by_force,
    'Recruitment_threshold_input': MN_recruitment_thresholds_by_excitatory_input,
    'Mean_firing_rate': MN_mean_firing_rates,
    'STD_firing_rate': MN_std_firing_rates,
    'Soma_size': motoneuron_soma_diameters,
    'Recruitment_threshold_input_clean': MN_recruitment_thresholds_by_excitatory_input_clean_MVC,
    'Recruitment_threshold_input_ratio': MN_recruitment_thresholds_by_excitatory_input_ratio,
    'Probability_of_being_sampled': sampling_probability_dsitribution_to_save,
    'Inhibition_weight_(1st_inhibitory_input)': MN_inhibition_weights[0],
})
new_filename = f'Individual_MUs_results.csv'
save_file_path = os.path.join(new_directory, new_filename)
RT_mean_DR_dataframe.to_csv(save_file_path, index=False)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Create a Linear Regression model
model = LinearRegression()
# Fit the model
model.fit(MN_recruitment_thresholds_by_force_only_selected_idx, MN_mean_firing_rates_only_selected_idx)
# Get the slope (coefficient)
slope = model.coef_[0]
# Predict the Y values using the fitted model
Y_pred = model.predict(MN_recruitment_thresholds_by_force_only_selected_idx)
# Calculate R squared
r_squared = r2_score(MN_mean_firing_rates_only_selected_idx, Y_pred)
plt.figure()
plt.scatter(MN_recruitment_thresholds_by_force_only_selected_idx,MN_mean_firing_rates_only_selected_idx,s=70,alpha=0.35,color='C1')
plt.plot(MN_recruitment_thresholds_by_force_only_selected_idx, Y_pred,color='C3',alpha=0.5, linewidth=3, label=f'Linear regression (slope = {np.round(slope[0]*100)/100}; R² = {np.round(r_squared*100)/100})')
plt.xlabel('Recruitment threshold (% MVC)')
plt.ylabel("Mean firing rate on the plateau (pps)")
plt.ylim(0,(np.ceil(max(MN_mean_firing_rates)/10)*10)+1)
plt.title("Recruitment threshold (in % MVC) to mean discharge rate relationship" + plot_title_suffix)
plt.legend()
plt.xlim(0,target_force_level)
new_filename = f'RT_force_to_mean_DR_relationship.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Create a Linear Regression model
model = LinearRegression()
# Fit the model
model.fit(MN_recruitment_thresholds_by_input_only_selected_idx, MN_mean_firing_rates_only_selected_idx)
# Get the slope (coefficient)
slope = model.coef_[0]
# Predict the Y values using the fitted model
Y_pred = model.predict(MN_recruitment_thresholds_by_input_only_selected_idx)
# Calculate R squared
r_squared = r2_score(MN_mean_firing_rates_only_selected_idx, Y_pred)
plt.figure()
plt.scatter(MN_recruitment_thresholds_by_input_only_selected_idx,MN_mean_firing_rates_only_selected_idx,s=70,alpha=0.35,color=[1,0.7,0])
plt.plot(MN_recruitment_thresholds_by_input_only_selected_idx, Y_pred, color=[0.8,0.25,0],alpha=0.5, linewidth=3, label=f'Linear regression (slope = {np.round(slope[0]*100)/100}; R² = {np.round(r_squared*100)/100})')
plt.xlabel('Recruitment threshold (excitatory input, milliSiemens)')
plt.ylabel("Mean firing rate on the plateau (pps)")
plt.ylim(0,(np.ceil(max(MN_mean_firing_rates)/10)*10)+1)
plt.title("Recruitment threshold (excitatory input) to mean discharge rate relationship" + plot_title_suffix)
plt.legend()
new_filename = f'RT_input_to_mean_DR_relationship.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


mean_inhib_weights_of_active_MUs = 0
if nb_inhibitory_input > 0:
    for inhibiti in range(nb_inhibitory_input):
        mean_inhib_weights_of_active_MUs += np.mean(MN_inhibition_weights[inhibiti][selected_motor_units])
    mean_inhib_weights_of_active_MUs = mean_inhib_weights_of_active_MUs / nb_inhibitory_input

### Force output
force_output_window_of_analyzis = output_force[samples_for_analyzis]
force_output_error_window_of_analyzis = np.abs(output_force[samples_for_analyzis] - target_force[samples_for_analyzis])
mean_force_error_window_of_analyzis = np.mean(force_output_error_window_of_analyzis)
mean_force_output_window_of_analyzis = np.mean(force_output_window_of_analyzis)
std_force_output_window_of_analyzis = np.std(force_output_window_of_analyzis)

### Mean excitatory input during the plateau
# np.mean(final_mean_excit_input[samples_for_analyzis])

### Main results
main_results_dataframe = pd.DataFrame({
    'Simulation_name': sim_name,
    'Window_of_analyzis_duration': (window_analyzis_end-window_analyzis_begin)/fsamp,
    'Ground_truth_PCI_ignoring_inhibition': [PCI_ground_truth_without_inhib], #[] to make sure at least the first value is an array
    'PCSI_estimation': PCI_estimated,
    'VAF_for_PC1': cumulative_explained_variance[0],
    'Slope_of_RT_mean_DR_relationship': slope[0],
    'Nb_of_continously_active_MUs': len(valid_MUs_idx),
    'Nb_of_sampled_MUs_for_analyzis': len(selected_motor_units),
    'Mean_DR': np.mean(MN_mean_firing_rates_only_selected_idx),
    'std_DR': np.std(MN_mean_firing_rates_only_selected_idx),
    'Mean_RT': np.mean(MN_recruitment_thresholds_by_force_only_selected_idx),
    'std_RT': np.std(MN_recruitment_thresholds_by_force_only_selected_idx),
    '0-5hz power of common input': common_noise_power_intergal_0_5_hz,
    '0-5hz power of inhibition': inhibitory_input_power_integral_0_5_hz_mean,
    '0-5hz power of independent excitatory noise': independent_excit_noise_power_integral_0_5_hz_mean,
    'Mean weight of inhibition': mean_inhib_weights_of_active_MUs,
    'Mean torque': mean_force_output_window_of_analyzis,
    'Mean torque error': mean_force_error_window_of_analyzis,
    'Mean excitatory input on plateau': np.mean(final_mean_excit_input[samples_for_analyzis]),
    'Torque variability (torque std)': std_force_output_window_of_analyzis
    })
new_filename = f'Main_results.csv'
save_file_path = os.path.join(new_directory, new_filename)
main_results_dataframe.to_csv(save_file_path, index=False)

### Cumulative Explained Variance
VAF_cumsum_dataframe = pd.DataFrame({
    'PC': np.arange(len(cumulative_explained_variance))+1,
    'Cumulative_VAF': cumulative_explained_variance
    })
new_filename = f'Cumulative_VAF_PCA.csv'
save_file_path = os.path.join(new_directory, new_filename)
VAF_cumsum_dataframe.to_csv(save_file_path, index=False)

In [ ]:
plt.figure(figsize=(30,10))

# Plot mean firing rates as a curve according to MN index
plt.subplot(231)
plt.plot(firing_rates, alpha = 0.5, linewidth = 2, label = "all MNs", color='C0')
plt.scatter(selected_motor_units, firing_rates[selected_motor_units], color = 'royalblue', alpha = 0.5, linewidth = 2, label = "Selected MNs")
plt.ylim(-0.5,np.ceil(np.max(firing_rates[selected_motor_units]))+0.5)
plt.xlim(0-np.round(nb_motoneurons_full_pool/10),nb_motoneurons_full_pool+np.round(nb_motoneurons_full_pool/10))
plt.ylabel("Mean firing rate (pps)")
plt.xlabel(f"MN index (0 = smallest MU; {nb_motoneurons_full_pool} = largest MU)")
plt.title("Mean firing rates of simulated MNs - MN index")
plt.legend()

plt.subplot(234)
plt.plot(motoneuron_soma_diameters,firing_rates, alpha = 0.5, linewidth = 2, label = "all MNs", color='C0')
plt.scatter(motoneuron_soma_diameters[selected_motor_units], firing_rates[selected_motor_units], color = 'royalblue', alpha = 0.5, linewidth = 2, label = "Selected MNs")
plt.ylim(-0.5,np.ceil(np.max(firing_rates[selected_motor_units]))+0.5)
plt.xlim(min_soma_diameter-np.round(min_soma_diameter/10),max_soma_diameter+np.round(min_soma_diameter/10))
plt.ylabel("Mean firing rate (pps)")
plt.xlabel(f"Motoneuron size (soma diameter in micrometers)")
plt.title("Mean firing rates of simulated MNs - MN size")
plt.legend()

plt.subplot(232)
plt.plot(MN_recruitment_thresholds_by_force, alpha = 0.5, linewidth = 2, label = "all MNs", color='C1')
plt.scatter(selected_motor_units, MN_recruitment_thresholds_by_force[selected_motor_units], color = 'C1', alpha = 0.5, linewidth = 2, label = "Selected MNs")
plt.ylim(0,35)
plt.xlim(0-np.round(nb_motoneurons_full_pool/10),nb_motoneurons_full_pool+np.round(nb_motoneurons_full_pool/10))
plt.ylabel("Recruitment threshold (% MVC)")
plt.xlabel(f"MN index (0 = smallest MU; {nb_motoneurons_full_pool} = largest MU)")
plt.title("Recruitment thresholds (by force) of simulated MNs - MN index")
plt.legend()

plt.subplot(235)
plt.plot(motoneuron_soma_diameters,MN_recruitment_thresholds_by_force, alpha = 0.5, linewidth = 2, label = "all MNs", color='C1')
plt.scatter(motoneuron_soma_diameters[selected_motor_units], MN_recruitment_thresholds_by_force[selected_motor_units], color = 'C1', alpha = 0.5, linewidth = 2, label = "Selected MNs")
plt.ylim(0,35)
plt.xlim(min_soma_diameter-np.round(min_soma_diameter/10),max_soma_diameter+np.round(min_soma_diameter/10))
plt.ylabel("Recruitment threshold (% MVC)")
plt.xlabel(f"Motoneuron size (soma diameter in micrometers)")
plt.title("Recruitment thresholds (by force) of simulated MNs - MN size")
plt.legend()

plt.subplot(233)
plt.plot(MN_recruitment_thresholds_by_excitatory_input, alpha = 0.5, linewidth = 2, label = "all MNs", color=[1,0.7,0])
plt.scatter(selected_motor_units, MN_recruitment_thresholds_by_excitatory_input[selected_motor_units], color = [1,0.7,0], alpha = 0.5, linewidth = 2, label = "Selected MNs")
plt.xlim(0-np.round(nb_motoneurons_full_pool/10),nb_motoneurons_full_pool+np.round(nb_motoneurons_full_pool/10))
plt.ylabel("Recruitment threshold (excitatory input, in milliSiemens)")
plt.xlabel(f"MN index (0 = smallest MU; {nb_motoneurons_full_pool} = largest MU)")
plt.title("Recruitment thresholds (by input) of simulated MNs - MN index")
plt.legend()

plt.subplot(236)
plt.plot(motoneuron_soma_diameters,MN_recruitment_thresholds_by_excitatory_input, alpha = 0.5, linewidth = 2, label = "all MNs", color=[1,0.7,0])
plt.scatter(motoneuron_soma_diameters[selected_motor_units], MN_recruitment_thresholds_by_excitatory_input[selected_motor_units], color = [1,0.7,0], alpha = 0.5, linewidth = 2, label = "Selected MNs")
plt.xlim(min_soma_diameter-np.round(min_soma_diameter/10),max_soma_diameter+np.round(min_soma_diameter/10))
plt.ylabel("Recruitment threshold (excitatory input, in milliSiemens)")
plt.xlabel(f"Motoneuron size (soma diameter in micrometers)")
plt.title("Recruitment thresholds (by input) of simulated MNs - MN size")
plt.legend()

plt.suptitle("Firing rates and recruitment thresholds")

new_filename = f'MN_RT_and_DR_properties.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()